In [4]:
from pymongo import MongoClient
import os
import yaml
from pprint import pprint

In [5]:
import sys
import os
# Agregar el directorio 'src' al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

In [6]:
from utils.mongo_client import MongoClient
mongo = MongoClient()

2025-10-02 19:10:09.308 | INFO     | utils.mongo_client:_check_connection:68 - [MongoClient] Checking MongoDB connection...
2025-10-02 19:10:09.973 | INFO     | utils.mongo_client:_check_connection:70 - [MongoClient] MongoDB connection successful.
2025-10-02 19:10:09.974 | INFO     | utils.mongo_client:_verify_database_exists:34 - [MongoClient] Checking database existence...
2025-10-02 19:10:10.026 | INFO     | utils.mongo_client:_verify_database_exists:39 - [MongoClient] Database 'multimodal-rag' exists.
2025-10-02 19:10:10.028 | INFO     | utils.mongo_client:_verify_collection_exists:43 - [MongoClient] Checking collection existence...
2025-10-02 19:10:10.083 | INFO     | utils.mongo_client:_verify_collection_exists:48 - [MongoClient] Collection 'multimodal-rag' exists.
2025-10-02 19:10:10.085 | INFO     | utils.mongo_client:_verify_vector_index_exists:52 - [MongoClient] Checking vector index existence...
2025-10-02 19:10:10.206 | INFO     | utils.mongo_client:_verify_vector_index_exi

#### Vector Query

In [12]:
# Step -1: Create the Vector Index (Vector Search Model)
from pymongo.operations import SearchIndexModel

        {
        "fields":[
            {
            "type": "vector",
            "path": "<field-to-index>",
            "numDimensions": <number-of-dimensions>,
            "similarity": "euclidean | cosine | dotProduct",
            "quantization": "none | scalar | binary",
            "hnswOptions": {
                "maxEdges": <number-of-connected-neighbors>,
                "numEdgeCandidates": <number-of-nearest-neighbors>
            }
        ]
        }

In [13]:
search_index_model = SearchIndexModel(
    definition= {
        "fields" : [
            {
                "type": "vector",
                "path": "embedding_text",
                "numDimensions": 1024,
                "similarity": "cosine"

            }
        ]
    },
    name = "vector_search_index",
    type= "vectorSearch",
)

In [14]:
mongo.collection.create_search_index(search_index_model)

'vector_search_index'

In [15]:
# Listar los índices existentes en la colección
try:
    indexes = mongo.collection.list_search_indexes()
    print("Índices existentes:")
    for index in indexes:
        print(index)
except Exception as e:
    print(f"Error al obtener los índices: {e}")

Índices existentes:
{'id': '68d8328ef1f92a3191a1bf61', 'name': 'vector_text', 'type': 'vectorSearch', 'status': 'READY', 'queryable': True, 'latestDefinitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 9, 27, 18, 53, 2, 876000)}, 'latestDefinition': {'fields': [{'type': 'vector', 'path': 'embedding_text', 'numDimensions': 1024, 'similarity': 'cosine', 'quantization': 'none'}]}, 'statusDetail': [{'hostname': 'atlas-jz7esd-shard-00-00', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 9, 27, 18, 53, 2)}, 'definition': {'fields': [{'type': 'vector', 'path': 'embedding_text', 'numDimensions': 1024, 'similarity': 'cosine'}]}}}, {'hostname': 'atlas-jz7esd-shard-00-01', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 9, 27, 18, 53, 2)}, 'definition': 

In [27]:
# Eliminar un índice de búsqueda específico
try:
    mongo.db.command("dropSearchIndex", mongo.collection.name, name="vector_search_index")
    print("Índice vectorial eliminado correctamente.")
except Exception as e:
    print(f"Error al eliminar el índice vectorial: {e}")


Error al eliminar el índice vectorial: Search index multimodal-rag.multimodal-rag.vector_search_index cannot be found, full error: {'ok': 0.0, 'errmsg': 'Search index multimodal-rag.multimodal-rag.vector_search_index cannot be found', 'code': 27, 'codeName': 'IndexNotFound', '$clusterTime': {'clusterTime': Timestamp(1759013601, 1), 'signature': {'hash': b'KR\x06\xc5\xd5\xd8\xd4>\xb4\xac\xa3T\xa9\xff\xa1\x9bd\xed\xf2\xd3', 'keyId': 7526364577701822470}}, 'operationTime': Timestamp(1759013601, 1)}


In [38]:
# Step - 2: query : Aggregate pipeline
# query_vector = [0.10, -0.60, 0.50, 0.85, -0.70, 0.30, 0.20, -0.60]
# {
#   "text": "An emotional story of friendship and hope inside a prison.",
#   "query_vector": [0.24, 0.65, -0.10, 0.92, 0.40, -0.50, 0.59, -0.02]
# }
queryVector =  [
    0.02576340362429619,
    0.009360621683299541,
    -0.02033718302845955,
    0.011193360202014446,
    0.02128172665834427,
    -0.038188766688108444,
    -0.007128851022571325,
    0.049111299216747284,
    -0.013366425409913063,
    0.014846711419522762,
    -0.012796849943697453,
    0.03550394997000694,
    -0.00006725909770466387,
    -0.012419819831848145,
    0.029432544484734535,
    -0.008461262099444866,
    -0.013255016878247261,
    -0.0009088408551178873,
    0.039597176015377045,
    0.010906979441642761,
    -0.03343667462468147,
    0.0009757004445418715,
    0.0030345814302563667,
    -0.04357939958572388,
    -0.005566384643316269,
    0.07443536818027496,
    -0.02892092615365982,
    0.006827964913100004,
    0.005490221548825502,
    0.020962992683053017,
    0.04500056058168411,
    0.03302237391471863,
    0.05896613746881485,
    -0.00860665924847126,
    0.007788133341819048,
    -0.03734836354851723,
    0.007959868758916855,
    -0.01728088967502117,
    -0.07997047901153564,
    0.04178342968225479,
    -0.000005240518021309981,
    -0.016460420563817024,
    -0.03937547281384468,
    0.015165407210588455,
    0.03956892713904381,
    0.018477683886885643,
    0.0008457311778329313,
    -0.06174836680293083,
    0.015991494059562683,
    -0.03805692121386528,
    0.013626977801322937,
    -0.018808778375387192,
    -0.009155192412436008,
    -0.01520315371453762,
    0.005390331149101257,
    0.006976805627346039,
    -0.028399480506777763,
    -0.01124759204685688,
    -0.03322833776473999,
    -0.006071718875318766,
    -0.03836987912654877,
    0.031160295009613037,
    -0.053107816725969315,
    -0.03128385916352272,
    -0.005247592460364103,
    -0.0033181807957589626,
    0.008368861861526966,
    -0.005556933581829071,
    0.03816976770758629,
    -0.05787196382880211,
    0.02442270889878273,
    0.010953166522085667,
    -0.010868392884731293,
    -0.011561363004148006,
    -0.0336347259581089,
    -0.006836547050625086,
    0.016888422891497612,
    0.0043161772191524506,
    0.005342505872249603,
    0.0017348796827718616,
    0.07460211962461472,
    0.008731174282729626,
    0.0127928601577878,
    -0.0035399587359279394,
    -0.021842677146196365,
    -0.005383252631872892,
    -0.005318572744727135,
    0.02319157123565674,
    -0.0012525304919108748,
    -0.010946272872388363,
    -0.022350499406456947,
    0.03295506536960602,
    -0.0037921147886663675,
    -0.02268597111105919,
    -0.005220592021942139,
    -0.01150692068040371,
    0.01727728731930256,
    -0.014419732615351677,
    -0.004124445375055075,
    0.029202720150351524,
    -0.021539712324738503,
    0.010598336346447468,
    0.016636580228805542,
    0.023824790492653847,
    0.01978749968111515,
    -0.02707015722990036,
    0.025422552600502968,
    0.003723237430676818,
    0.011311843991279602,
    -0.005028287880122662,
    0.009413096122443676,
    -0.0139675447717309,
    -0.009178501553833485,
    -0.00309150293469429,
    0.006422383710741997,
    -0.015174255706369877,
    -0.01247486099600792,
    -0.02163851633667946,
    0.028496243059635162,
    0.014381438493728638,
    0.018687855452299118,
    0.02987612970173359,
    0.022428292781114578,
    -0.02078140340745449,
    -0.002275675069540739,
    -0.006318459752947092,
    -0.02402265928685665,
    -0.0102431271225214,
    -0.02023814432322979,
    0.010001045651733875,
    0.0019474312430247664,
    0.035202790051698685,
    -0.03411136567592621,
    0.0037975329905748367,
    -0.0033773002214729786,
    0.004602823406457901,
    0.0010431240079924464,
    0.042582910507917404,
    -0.0109465466812253,
    -0.05686448514461517,
    0.027122752740979195,
    -0.01703203283250332,
    -0.01969733275473118,
    -0.012198014184832573,
    0.023366598412394524,
    -0.0371270552277565,
    0.01716090366244316,
    -0.01684349589049816,
    0.021154897287487984,
    0.0028967191465198994,
    0.0031821501906961203,
    0.016796370968222618,
    0.0038295320700854063,
    -0.007290436886250973,
    0.020124787464737892,
    -0.012503326870501041,
    0.032577551901340485,
    0.036008674651384354,
    0.002411692636087537,
    -0.006840021349489689,
    0.034493301063776016,
    -0.013272539712488651,
    0.034364935010671616,
    0.011801655404269695,
    0.011161447502672672,
    -0.019780335947871208,
    0.01498445775359869,
    0.02016756683588028,
    0.001967522082850337,
    0.011117909103631973,
    0.021107090637087822,
    -0.014288190752267838,
    0.003130940720438957,
    0.010372169315814972,
    0.015538543462753296,
    0.027082208544015884,
    0.017619812861084938,
    0.022155513986945152,
    0.04618105664849281,
    -0.012700177729129791,
    0.008427516557276249,
    0.010134868323802948,
    0.015086772851645947,
    0.004002757370471954,
    0.016740499064326286,
    0.006661849096417427,
    0.0011467100121080875,
    0.021392812952399254,
    0.00914442352950573,
    0.02748110145330429,
    -0.000286288297502324,
    -0.00528909545391798,
    0.015397215262055397,
    0.01307168323546648,
    0.02744423784315586,
    0.022038251161575317,
    -0.012780112214386463,
    0.00847757887095213,
    0.05072350800037384,
    -0.011333818547427654,
    -0.008807671256363392,
    -0.028200266882777214,
    -0.02178727276623249,
    -0.009339707903563976,
    -0.0033364591654390097,
    0.013935219496488571,
    -0.06489396095275879,
    -0.009916015900671482,
    -0.0023888428695499897,
    -0.002513668267056346,
    -0.016978314146399498,
    -0.00014256629219744354,
    -0.02580435387790203,
    0.018080173060297966,
    -0.026769526302814484,
    0.02341761440038681,
    -0.0055939266458153725,
    -0.01852431893348694,
    0.006577682681381702,
    0.024045871570706367,
    -0.00014198465214576572,
    -0.0062036276794970036,
    -0.0217679962515831,
    -0.023407306522130966,
    -0.012155583128333092,
    -0.04148222133517265,
    0.001740451087243855,
    0.026979373767971992,
    0.002103510545566678,
    -0.013427816331386566,
    -0.03663730248808861,
    0.05571247637271881,
    -0.0028803099412471056,
    -0.03921755403280258,
    -0.00979503896087408,
    0.023038586601614952,
    -0.019579779356718063,
    0.004529715050011873,
    0.020785488188266754,
    0.007717590779066086,
    -0.00035382615169510245,
    -0.04117989540100098,
    0.024490797892212868,
    -0.02133420668542385,
    -0.023816131055355072,
    0.02485610730946064,
    -0.0206521637737751,
    -0.00027791777392849326,
    -0.012578373774886131,
    -0.013978557661175728,
    -0.008014275692403316,
    -0.019380994141101837,
    0.018234770745038986,
    -0.0397210493683815,
    -0.01572423428297043,
    -0.004695048090070486,
    -0.022622190415859222,
    -0.014804193750023842,
    -0.02555665746331215,
    0.021870192140340805,
    0.03508877009153366,
    0.010382783599197865,
    0.001848551444709301,
    0.013870848342776299,
    -0.0007340482552535832,
    -0.001544668572023511,
    0.0006436305120587349,
    -0.03692564368247986,
    0.009919088333845139,
    0.023853804916143417,
    -0.013513264246284962,
    -0.019081540405750275,
    -0.01410249900072813,
    -0.012007288634777069,
    -0.03092356212437153,
    0.03380783274769783,
    0.0024249483831226826,
    0.007956805638968945,
    0.03572043031454086,
    -0.030810201540589333,
    0.02030579373240471,
    -0.024407804012298584,
    -0.02272137999534607,
    -0.02770889550447464,
    0.003704882925376296,
    0.030109485611319542,
    -0.007304370403289795,
    -0.01114481221884489,
    0.03162013366818428,
    -0.02035730890929699,
    -0.009969823062419891,
    0.009192649275064468,
    0.09735219180583954,
    -0.013476015999913216,
    -0.0019728646147996187,
    -0.004480048082768917,
    -0.011230689473450184,
    -0.09995593130588531,
    -0.005419634748250246,
    -0.0032952253241091967,
    0.015820758417248726,
    0.02920451946556568,
    0.00014649717195425183,
    0.010398944839835167,
    -0.007139043882489204,
    -0.004217384848743677,
    -0.02708258479833603,
    -0.028520725667476654,
    -0.11419124156236649,
    -0.011984992772340775,
    -0.0015402305871248245,
    0.0435648113489151,
    -0.016101911664009094,
    -0.0171633493155241,
    0.03132583200931549,
    -0.03038819506764412,
    -0.0660228282213211,
    0.006736293435096741,
    0.008033952675759792,
    0.031132133677601814,
    0.01784348115324974,
    0.03762274608016014,
    0.013883004896342754,
    0.035859059542417526,
    0.012342164292931557,
    -0.0182250514626503,
    0.010444343090057373,
    -0.060482531785964966,
    0.012644259259104729,
    -0.015504258684813976,
    0.002051803981885314,
    0.019002152606844902,
    -0.005273809656500816,
    0.030022630468010902,
    -0.028760937973856926,
    -0.00502996938303113,
    0.02422969974577427,
    0.05220157280564308,
    -0.00433014752343297,
    -0.01523615699261427,
    -0.019485564902424812,
    0.00030484190210700035,
    -0.014861336909234524,
    0.009660373441874981,
    0.008959639817476273,
    -0.04303009808063507,
    0.039036963135004044,
    -0.0111321322619915,
    0.005286280997097492,
    -0.01538815163075924,
    0.05030878633260727,
    -0.016136934980750084,
    0.0009887778433039784,
    0.02219865471124649,
    0.026504183188080788,
    0.053377822041511536,
    -0.0361151285469532,
    0.002478290582075715,
    -0.02309063822031021,
    -0.021455593407154083,
    -0.00723297568038106,
    0.009844343177974224,
    -0.02720431052148342,
    -0.008728881366550922,
    -0.024917300790548325,
    -0.008404183201491833,
    -0.03771243244409561,
    -0.018365509808063507,
    -0.015426612459123135,
    -0.016851508989930153,
    -0.029205111786723137,
    0.010633287020027637,
    -0.024600490927696228,
    -0.011017872020602226,
    -0.0081470413133502,
    -0.023382332175970078,
    -0.07534737139940262,
    -0.00019106785475742072,
    0.002030935138463974,
    -0.03230637311935425,
    0.04928071051836014,
    -0.017949428409337997,
    -0.010799333453178406,
    0.01631188951432705,
    -0.024347437545657158,
    0.028152232989668846,
    0.6448132395744324,
    -0.012804281897842884,
    0.004051920026540756,
    0.017905885353684425,
    -0.0035612862557172775,
    0.00876891240477562,
    0.03583814948797226,
    -0.009196248836815357,
    0.007320627104490995,
    -0.026435254141688347,
    -0.03879249840974808,
    -0.01895155943930149,
    0.026962703093886375,
    -0.01494997926056385,
    0.022670334205031395,
    -0.0034537806641310453,
    -0.017571400851011276,
    -0.011676420457661152,
    0.020172394812107086,
    -0.009777216240763664,
    0.03216412663459778,
    -0.04218504950404167,
    0.01909695938229561,
    -0.031851742416620255,
    -0.04931431636214256,
    -0.0001427680253982544,
    0.02442573942244053,
    0.024323591962456703,
    -0.01781047135591507,
    0.01117523293942213,
    -0.007879886776208878,
    -0.003835015231743455,
    -0.004792606923729181,
    -0.031874991953372955,
    -0.025574684143066406,
    -0.018883243203163147,
    -0.051822636276483536,
    -0.0280194990336895,
    -0.01624670997262001,
    0.024246204644441605,
    0.019025027751922607,
    -0.029953844845294952,
    -0.04674096405506134,
    -0.018993260338902473,
    -0.03479152172803879,
    -0.028367122635245323,
    -0.003701327368617058,
    0.00525760930031538,
    0.014543244615197182,
    -0.0201478973031044,
    0.024354154244065285,
    0.009594841860234737,
    -0.03660377115011215,
    0.02472779154777527,
    -0.02331729233264923,
    -0.008503664284944534,
    0.005679755937308073,
    -0.008175069466233253,
    0.01554773934185505,
    0.011933579109609127,
    0.018469616770744324,
    0.024591971188783646,
    0.01061598863452673,
    0.013434870168566704,
    -0.006025230046361685,
    -0.007718589622527361,
    -0.032225094735622406,
    -0.014881405048072338,
    -0.014060067012906075,
    0.04128642752766609,
    0.054128844290971756,
    0.04975879192352295,
    -0.018672673031687737,
    0.006878660060465336,
    0.016192255541682243,
    -0.040669627487659454,
    0.025295959785580635,
    0.029546914622187614,
    0.004641273990273476,
    0.023989403620362282,
    -0.012928943149745464,
    -0.034332968294620514,
    0.013189587742090225,
    -0.018134918063879013,
    0.02054973505437374,
    -0.01132361963391304,
    0.006138592027127743,
    -0.019910158589482307,
    0.005541922524571419,
    -0.015311066061258316,
    0.012894947081804276,
    -0.03297040984034538,
    0.0074746436439454556,
    -0.008420092053711414,
    0.00888034887611866,
    0.009639188647270203,
    -0.0004800869501195848,
    -0.04224351793527603,
    -0.015091516077518463,
    -0.023693963885307312,
    -0.007592594251036644,
    0.015828050673007965,
    -0.010404320433735847,
    0.012935257516801357,
    0.00886059645563364,
    -0.02282288298010826,
    -0.024074450135231018,
    -0.041560277342796326,
    -0.042659878730773926,
    -0.004415848758071661,
    -0.04373185336589813,
    0.023366574198007584,
    0.003858029842376709,
    0.028508150950074196,
    0.01858212985098362,
    0.014929689466953278,
    -0.023289931938052177,
    0.023305153474211693,
    0.023594187572598457,
    -0.004369748756289482,
    -0.014699939638376236,
    -0.00314516993239522,
    -0.007179854437708855,
    -0.028157860040664673,
    -0.025691762566566467,
    -0.027960678562521935,
    -0.03574225306510925,
    0.0014075788203626871,
    -0.008454643189907074,
    0.01614418625831604,
    -0.0016095610335469246,
    0.02772979438304901,
    -0.03447086364030838,
    -0.032701704651117325,
    0.025734232738614082,
    -0.018149113282561302,
    0.03314801678061485,
    0.019652919843792915,
    -0.013952851295471191,
    0.03982551395893097,
    -0.02137908712029457,
    -0.06829309463500977,
    -0.02446196973323822,
    0.011091405525803566,
    -0.01890302635729313,
    0.0015815222868695855,
    0.015164713375270367,
    -0.02549273520708084,
    -0.010756800882518291,
    -0.03062410280108452,
    0.025986094027757645,
    -0.007305746898055077,
    -0.002884078770875931,
    -0.011517432518303394,
    0.03745114803314209,
    -0.023046128451824188,
    0.015685910359025,
    0.04263097420334816,
    -0.0333872027695179,
    -0.008650489151477814,
    -0.025011951103806496,
    -0.019971219822764397,
    -0.04456835612654686,
    0.03674701228737831,
    0.014918562956154346,
    0.0063456520438194275,
    -0.010381965897977352,
    0.011000635102391243,
    -0.0381711944937706,
    0.005346497055143118,
    0.022700771689414978,
    -0.0005490042385645211,
    -0.002537888940423727,
    0.03755445033311844,
    0.018007734790444374,
    -0.0061761680990457535,
    -0.0330129899084568,
    0.004065109416842461,
    -0.005576769821345806,
    0.0025390007067471743,
    0.043166060000658035,
    0.000978470197878778,
    -0.031402215361595154,
    0.017119521275162697,
    -0.03129764273762703,
    -0.010114689357578754,
    -0.034905508160591125,
    0.016752462834119797,
    0.06288216263055801,
    -0.020105015486478806,
    -0.017845969647169113,
    0.045407310128211975,
    -0.009825111366808414,
    -0.020632272586226463,
    0.02845306135714054,
    -0.013152841478586197,
    -0.036721017211675644,
    -0.03285600244998932,
    -0.010666457936167717,
    -0.005757756065577269,
    0.017366111278533936,
    0.034153979271650314,
    0.017666148021817207,
    0.032710544764995575,
    0.006174690555781126,
    -0.034177809953689575,
    -0.007765889633446932,
    0.02305423654615879,
    0.003663740586489439,
    0.008383197709918022,
    0.02162620984017849,
    -0.040723565965890884,
    0.015874402597546577,
    0.0031851944513618946,
    -0.0035379710607230663,
    -0.007572592236101627,
    0.011061453260481358,
    0.005664986092597246,
    0.022486751899123192,
    0.008171201683580875,
    -0.02822936698794365,
    -0.013747641816735268,
    0.04051363468170166,
    -0.010372827760875225,
    -0.0013048205291852355,
    0.022823749110102654,
    0.020852822810411453,
    -0.034064553678035736,
    -0.03764541819691658,
    -0.007180295884609222,
    -0.01174505241215229,
    -0.0013462210772559047,
    0.022166408598423004,
    0.001519324374385178,
    -0.008293184451758862,
    -0.03683987259864807,
    0.011479134671390057,
    0.028961731120944023,
    -0.0005523015279322863,
    0.009484007023274899,
    -0.007870785892009735,
    -0.001956252148374915,
    0.0015693294117227197,
    0.049530029296875,
    -0.0032341708429157734,
    0.018136613070964813,
    -0.013226103037595749,
    0.014139294624328613,
    0.024619774892926216,
    -0.014749906025826931,
    0.009788925759494305,
    -0.01721467822790146,
    -0.013018216006457806,
    0.006283655297011137,
    0.0036417986266314983,
    0.00030952601809985936,
    -0.003832092508673668,
    0.026555247604846954,
    -0.02049468457698822,
    0.01135775912553072,
    0.017982838675379753,
    0.011341546662151814,
    0.00016575530753470957,
    0.020397072657942772,
    -0.002777099609375,
    -0.02748158946633339,
    -0.0426119789481163,
    -0.04151052236557007,
    -0.0077925934456288815,
    0.009619960561394691,
    -0.002560954075306654,
    -0.006154120899736881,
    -0.013065410777926445,
    -0.02836572378873825,
    -0.022073278203606606,
    -0.012692022137343884,
    0.024665748700499535,
    0.0030168751254677773,
    0.029935136437416077,
    -0.03461955487728119,
    -0.02592570148408413,
    -0.014702784828841686,
    -0.028624020516872406,
    -0.01794438809156418,
    -0.0059300013817846775,
    -0.01922779344022274,
    -0.03520677238702774,
    -0.02033676765859127,
    0.013459435664117336,
    -0.019469257444143295,
    0.031234707683324814,
    -0.01815115101635456,
    -0.03345804288983345,
    -0.000658813863992691,
    -0.014600561931729317,
    0.022252051159739494,
    -0.038232557475566864,
    0.014466364867985249,
    -0.004533635918051004,
    0.0045053088106215,
    0.025366755202412605,
    0.011579213663935661,
    -0.00346272811293602,
    0.0009981542825698853,
    0.0011469038436189294,
    -0.014612500555813313,
    -0.018049664795398712,
    0.03849506750702858,
    -0.017212940379977226,
    -0.014703621156513691,
    -0.013456729240715504,
    -0.03644564747810364,
    -0.02918141707777977,
    -0.0316457636654377,
    -0.03166470304131508,
    -0.023129820823669434,
    -0.01900576613843441,
    0.0013233530335128307,
    -0.01305182371288538,
    0.004625529516488314,
    0.003912927582859993,
    0.007523528765887022,
    -0.028870439156889915,
    -0.006826979573816061,
    0.003299232106655836,
    -0.0024480707943439484,
    -0.03137786313891411,
    0.010968580842018127,
    -0.008181717246770859,
    -0.005518621299415827,
    -0.03962026908993721,
    0.012059098109602928,
    0.0008612315286882222,
    -0.0014438298530876637,
    -0.021030835807323456,
    -0.016010306775569916,
    0.04664130508899689,
    -0.003241228638216853,
    0.014166724868118763,
    0.002598751336336136,
    -0.0205841064453125,
    -0.008499129675328732,
    0.028082337230443954,
    -0.009782764129340649,
    0.02022596076130867,
    0.018935810774564743,
    0.009601202793419361,
    0.03477118909358978,
    -0.009809383191168308,
    -0.017491662874817848,
    -0.007161957677453756,
    0.004828211385756731,
    -0.026506474241614342,
    0.0034036755096167326,
    -0.006954774260520935,
    0.003526628715917468,
    0.006880367640405893,
    0.014703872613608837,
    0.02301829867064953,
    -0.011856863275170326,
    -0.030833221971988678,
    -0.005993707105517387,
    0.010552647523581982,
    -0.004647075664252043,
    -0.005416007246822119,
    -0.022424791008234024,
    -0.024143340066075325,
    -0.023033536970615387,
    0.0014378265477716923,
    0.0287193413823843,
    0.03318685665726662,
    0.018163951113820076,
    -0.01397419348359108,
    0.006501735188066959,
    -0.015377296134829521,
    -0.01114120613783598,
    0.0261837188154459,
    -0.03826364129781723,
    0.010766930878162384,
    -0.05469156429171562,
    -0.008750070817768574,
    -0.01852543279528618,
    -0.021011829376220703,
    -0.020220553502440453,
    0.01881132274866104,
    0.00937708094716072,
    -0.030672594904899597,
    0.024049609899520874,
    0.0029448000714182854,
    0.03621404990553856,
    -0.002100785495713353,
    0.012987908907234669,
    -0.006997353862971067,
    -0.02614229917526245,
    -0.08983953297138214,
    0.02872265689074993,
    -0.008367818780243397,
    -0.04418252408504486,
    -0.01113069150596857,
    0.0014626838965341449,
    -0.02951202169060707,
    -0.017615271732211113,
    -0.03593023493885994,
    -0.036528412252664566,
    0.02237449213862419,
    0.004717367235571146,
    0.0016729853814467788,
    0.005005748011171818,
    -0.012248080223798752,
    -0.009443654678761959,
    -0.05172432214021683,
    0.019839957356452942,
    -0.015575030818581581,
    0.021591322496533394,
    0.0024657966569066048,
    -0.024034371599555016,
    -0.03509053215384483,
    -0.01576032117009163,
    -0.002633293392136693,
    0.006019108463078737,
    0.00453940499573946,
    -0.007498419843614101,
    0.00521986186504364,
    -0.020388640463352203,
    0.02928153984248638,
    -0.007521128747612238,
    -0.03562508523464203,
    0.0199259202927351,
    -0.01289540994912386,
    0.0381603017449379,
    0.011393990367650986,
    -0.017138700932264328,
    0.010796334594488144,
    0.028562448918819427,
    0.013788524083793163,
    -0.018012210726737976,
    -0.006556098349392414,
    -0.00812698807567358,
    -0.024332845583558083,
    -0.014723449945449829,
    0.010739491321146488,
    0.007583132945001125,
    -0.01909586228430271,
    0.02487928234040737,
    -0.03576989099383354,
    0.006388066802173853,
    -0.02820558287203312,
    -0.0032499944791197777,
    -0.0029075422789901495,
    0.023768870159983635,
    0.026730533689260483,
    -0.028923138976097107,
    -0.0004206825396977365,
    0.012813044711947441,
    0.00024272134760394692,
    0.041817985475063324,
    -0.0030513680540025234,
    -0.020277244970202446,
    -0.02723880112171173,
    -0.004820888862013817,
    0.0027992711402475834,
    -0.039770111441612244,
    0.08072991669178009,
    -0.049829933792352676,
    -0.0415266714990139,
    -0.001111558056436479,
    0.000994521426036954,
    0.01978222094476223,
    0.010958556085824966,
    0.006988699082285166,
    -0.004698527976870537,
    -0.004720486234873533,
    0.027946900576353073,
    -0.008624181151390076,
    0.006586144678294659,
    -0.004823575261980295,
    0.030527086928486824,
    0.045147839933633804,
    0.038644276559352875,
    -0.026658305898308754,
    -0.007919076830148697,
    0.0058016348630189896,
    -0.0051177628338336945,
    0.010355718433856964,
    -0.0478425957262516,
    -0.007833234034478664,
    -0.012709389440715313,
    0.050847701728343964,
    0.020785341039299965,
    0.0040108272805809975,
    -0.015538375824689865,
    0.0020645686890929937,
    -0.002779625589028001,
    0.0014104186557233334,
    0.02417730912566185,
    0.024730412289500237,
    -0.030202355235815048,
    0.009654062800109386,
    0.01555011235177517,
    0.033060818910598755,
    -0.01819702237844467,
    0.017094556242227554,
    0.0021855072118341923,
    0.008402319625020027,
    -0.012262008152902126,
    -0.01974848471581936,
    -0.02810058556497097,
    0.011715810745954514,
    -0.028251197189092636,
    -0.0000718816154403612,
    0.017985820770263672,
    0.007106769364327192,
    -0.008434581570327282,
    -0.03125210478901863,
    0.0354117751121521,
    0.00507989851757884,
    0.004426690749824047,
    -0.07330501079559326,
    0.0053094192408025265,
    -0.0032038497738540173,
    -0.015576739795506,
    0.02974172681570053,
    0.001854629721492529,
    -0.0010176164796575904,
    0.023765889927744865,
    -0.005481339991092682,
    0.006455948110669851,
    0.02641518972814083,
    -0.03806645795702934,
    0.013093355111777782,
    -0.03158311918377876,
    0.06204185634851456,
    -0.0015109231462702155,
    -0.03030509687960148,
    0.01407048013061285,
    0.0043593659065663815,
    0.016572818160057068,
    -0.007800559978932142,
    0.009557773359119892,
    -0.007851986214518547,
    -0.012840238399803638,
    -0.01705441065132618,
    0.024707410484552383,
    -0.024530481547117233,
    0.007365829776972532,
    -0.04629048705101013,
    -0.02280888892710209,
    0.04717263951897621,
    0.0003944224154110998,
    -0.03947249799966812,
    0.03477635234594345,
    -0.010619053617119789,
    -0.018989380449056625,
    0.057626478374004364,
    0.06160486489534378,
    -0.0360841266810894,
    -0.018118536099791527,
    0.003376765875145793,
    -0.016673360019922256,
    0.013994862325489521,
    -0.004812228959053755,
    0.011885282583534718,
    0.012784578837454319,
    -0.02171475626528263,
    0.032173991203308105,
    0.009060835465788841,
    0.0071268584579229355,
    0.01673349365592003,
    -0.0036206026561558247,
    0.0015520830638706684,
    -0.004157375544309616,
    0.006929805502295494,
    0.003873577807098627,
    0.008798332884907722,
    0.006964343134313822,
    0.015489994548261166,
    0.027583401650190353,
    0.02482808567583561,
    0.01666521653532982,
    0.02236420288681984,
    0.033634524792432785,
    0.009649490937590599,
    -0.012197011150419712,
    -0.02095147967338562,
    0.005341438576579094,
    0.029818719252943993,
    -0.019233854487538338,
    0.008012219332158566,
    0.0055983602069318295,
    -0.019604535773396492,
    -0.009408898651599884,
    -0.012113923206925392,
    0.003505139611661434,
    0.01438179425895214,
    -0.03004458360373974,
    -0.008466861210763454,
    0.019174832850694656,
    -0.01866641640663147,
    -0.008146160282194614,
    -0.004380932077765465,
    -0.03274056315422058,
    -0.01818198896944523,
    -0.016184397041797638,
    0.01741405948996544,
    -0.018423333764076233,
    0.01150459237396717,
    -0.0002537621185183525,
    0.05398155748844147,
    -0.008527988567948341,
    0.0210020300000906,
    0.015955166891217232,
    0.02798505127429962,
    0.005438333377242088,
    -0.0009959653252735734,
    0.01995074935257435,
    0.056001439690589905,
    -0.015231525525450706,
    -0.048086605966091156,
    -0.01935381256043911,
    -0.04045485332608223,
    -0.009730351157486439,
    0.004301542416214943,
    -0.04464055970311165,
    -0.03200617432594299,
    -0.019650442525744438,
    0.011665351688861847,
    0.006438617594540119,
    0.013408774510025978,
    0.0290853139013052,
    0.009834802709519863,
    -0.025728892534971237,
    0.00033818400697782636,
    0.017040474340319633,
    0.014014098793268204,
    0.019917843863368034,
    -0.006527909077703953,
    -0.022392569109797478,
    0.012937652878463268
  
]

In [55]:
# vector search aggreagate pipeline
vectorsearch_pipeline = [
    # Stage - 1
   
  {
    "$vectorSearch": {
      "exact": True,
      "index": "vector_search_index",
      "limit": 10,
      "path": "embedding_text",
      "queryVector": queryVector
    }
  },
  {
      "$group": {
          "_id": "$source_file",
          "source_file": {"$first": "$source_file"},
          "text": {"$first": "$text"},
          "embedding_text": {"$first": "$embedding_text"},
          "score": {"$max": {"$meta": "vectorSearchScore"}}
      }
  },
  {
    "$project": {
      "_id": 0,
      "project_id": 1,
      "doc_type" : 1,
      "source_file": 1,
      "text": 1,
      "score": 1
    }
  },
  {
      "$sort": {"score": -1}
  },
  {
      "$limit": 5
  }

]

results = mongo.collection.aggregate(vectorsearch_pipeline)
# print the results
pprint(list(results))

[{'score': 1.0,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': '- **characteristics (caractersticas)**: must resist concrete and '
          '200 kg/m2 construction loads. - **tolerances (tolerancias)**: '
          'verticality (6 mm in 3m, 20mm total), alignment (6 mm in 3m, 10mm '
          'total), slabs (-5mm, +10mm), holes (5mm). - **stripping '
          '(desencofrado)**: minimum stripping times: vertical forms (10 hrs), '
          'beam bottoms (8 days), beam shores (15 days), slab bottoms (5 '
          'days), slab shores (11 days) for spans > 5m. - **6. albaileria '
          '(masonry)**: (for basements and perimeter walls only)\n'
          '- **masonry unit (unidad de albailera)**: clay or calcium-silicate '
          'solid/perforated as per plans annex 1 standards

In [23]:
queryVector = [
-0.8165319561958313, 0.33355095982551575, -0.710317075252533,     
                             0.4443209171295166, 0.576479434967041, 0.6274969577789307, 0.13935552537441254, 1.104414939880371, -0.3690758943557739, -0.7461910247802734, 0.4223950207233429, -0.18842484056949615,    
                             0.1361413151025772, -0.6563618183135986, 0.015042862854897976, 0.6899219751358032, 0.21376964449882507, -0.5659258365631104, -0.6797213554382324, -0.9156848788261414, 0.4524659812450409,
                             -1.6553491353988647, -0.17826230823993683, 0.28509843349456787, -0.7667925953865051, -0.14726674556732178, 0.5777832865715027, -0.30930086970329285, -0.6548582315444946,
                             0.41829583048820496, 0.5943952798843384, -0.6843632459640503, 0.4915415942668915, -0.9390547275543213, -0.6103430986404419, 0.1160895898938179, 0.2791438102722168, 0.18937262892723083,  
                             -0.8802141547203064, 0.2649594247341156, 0.30245786905288696, 0.49215155839920044, 0.4465184807777405, -0.33670833706855774, 0.5399115681648254, 0.07703713327646255, 0.3501838743686676, 
                             -0.8114666938781738, 0.2847922146320343, -0.5580542683601379, -1.3140466213226318, -0.3620307445526123, -0.08103134483098984, -0.6684709191322327, -0.07857052981853485,
                             -0.17527145147323608, -0.7124404907226562, -0.7458900213241577, -0.52301025390625, 0.4659794270992279, -0.9585422277450562, 0.47894781827926636, -0.07554604858160019, 0.3052913546562195,
                             0.0896499827504158, 1.15147066116333, 0.8359463214874268, 0.4300771653652191, 0.6631344556808472, -0.8437557816505432, -0.22681701183319092, -0.5424937009811401, -0.25804927945137024,   
                             -0.26224127411842346, -0.65282142162323, 0.7536718845367432, 0.5365412831306458, -1.0346890687942505, 0.13130830228328705, 0.6003758907318115, 0.45732659101486206, -0.2843587100505829,  
                             0.34279629588127136, 0.3648448884487152, -0.1409672647714615, 0.0630388855934143, -1.088192105293274, 0.9164182543754578, -0.4736393392086029, 0.4521534740924835, -0.6265469193458557,   
                             0.3513198792934418, -0.5794391632080078, 0.3994004726409912, -0.03130839765071869, -0.027633044868707657, -0.7597271800041199, 0.13970577716827393, 0.09782858192920685,
                             -0.21003104746341705, 1.0732395648956299, -0.44298821687698364, -0.1205049455165863, -0.6673112511634827, 0.4343016743659973, -0.04242733120918274, 0.15671676397323608,
                             -0.5304425954818726, 0.50539231300354, 0.11999021470546722, -0.496468186378479, -0.16305091977119446, -0.1563134342432022, 0.7744336724281311, 0.1020718365907669, -0.629250705242157,    
                             0.6433418393135071, -0.6578347086906433, 0.46045804023742676, 0.2009306401014328, 0.4693107604980469, 0.4054405093193054, 0.5299704670906067, -0.6380563974380493, -0.2632077634334564,   
                             -0.9176222085952759, -0.6166070103645325, 0.451561838388443, -0.4181797504425049, 0.740283191204071, 0.3504253625869751, 0.7679495811462402, 0.4133909344673157, -0.10929599404335022,    
                             -0.06269660592079163, -0.985852837562561, 0.4630261957645416, -0.25573012232780457, -0.13025012612342834, 0.02190849371254444, 0.7928504943847656, -0.0905369371175766,
                             -0.33216989040374756, -0.8412743210792542, -0.598192036151886, -0.8154960870742798, 0.9999369978904724, -0.790484607219696, 0.3133811056613922, 0.4272613525390625, 0.7034310698509216,   
                             0.8725700378417969, -0.18322235345840454, 0.23207733035087585, 0.8650190234184265, -1.0455206632614136, -0.7987379431724548, 0.5412012934684753, -0.25060343742370605,
                             0.06481847167015076, 1.2626625299453735, -0.19027204811573029, 1.0401896238327026, -0.5629823207855225, -0.6389135122299194, -0.8092586994171143, -0.2757764756679535, 1.3846055269241333,
                             0.7998999953269958, 0.6964121460914612, -0.6353060603141785, -0.5095622539520264, 0.40984994173049927, -0.10974348336458206, 0.4064212739467621, -0.3075941801071167, 0.4115482270717621, 
                             0.6373676061630249, -0.08481088280677795, -0.19340597093105316, 0.0009900950826704502, 0.9951579570770264, 0.2410636842250824, -1.1123367547988892, 0.22809475660324097,
                             0.47972580790519714, -0.2219991236925125, -0.5680616497993469, 0.45268797874450684, 0.10792194306850433, 0.037987422198057175, -0.6006259322166443, 0.17985564470291138,
                             -0.04190552607178688, 0.48637712001800537, -0.552636981010437, 0.09965354204177856, 0.4222029745578766, -0.0030939816497266293, -0.9809521436691284, -0.6197131872177124,
                             0.259841650724411, -0.8587894439697266, 0.2800167202949524, -0.6020995378494263, 0.6963226795196533, -0.6926200985908508, -0.48192620277404785, 0.4087767004966736, -0.12288499623537064, 
                             0.24988529086112976, 0.5464855432510376, 0.20555974543094635, 0.5294679403305054, -1.3168185949325562, 0.4612593948841095, -0.2150196135044098, -0.23348495364189148, -0.3331112563610077,
                             -0.17071272432804108, 0.14527787268161774, 0.22709296643733978, 1.126285195350647, 0.1484145224094391, -0.10449636727571487, 0.7570334076881409, 0.19714298844337463,
                             0.014145510271191597, 0.10117704421281815, -0.6449418663978577, -0.7746511697769165, -0.9697644114494324, -0.5349447727203369, -0.052735548466444016, 0.5417439937591553,
                             1.0771911144256592, 0.037907954305410385, -0.8052575588226318, -0.2853546440601349, -0.9411836266517639, 0.40986716747283936, 1.0075417757034302, 0.26977646350860596,
                             -0.6597520709037781, -0.4258771538734436, -0.3899713158607483, -0.1704469621181488, 1.0127959251403809, -0.3170057535171509, 0.6843637228012085, 1.0336132049560547, -0.542716920375824,  
                             -0.08660827577114105, -0.01424345001578331, -0.3792197108268738, 0.7141510844230652, 0.04456305503845215, -0.09805759787559509, 0.25574442744255066, 0.3762683868408203,
                             0.9265849590301514, 0.5405340194702148, -0.17797203361988068, 0.21242469549179077, 0.07289374619722366, -0.40246260166168213, -0.44133299589157104, 0.25817617774009705,
                             0.5584651827812195, 0.01375636551529169, -0.7326222062110901, 0.14247648417949677, -0.30816447734832764, -1.3053710460662842, 0.04785322770476341, -1.22830069065094, -0.370818167924881, 
                             -0.3949677646160126, 0.5460121035575867, 0.31030380725860596, 0.14978128671646118, -0.32082828879356384, 0.5661147236824036, -0.2630406618118286, 0.08570711314678192,
                             -0.24435344338417053, -0.1976565569639206, 0.2201092541217804, 1.1741949319839478, -0.11883939802646637, 0.698005735874176, -0.26329994201660156, 2.206780433654785, 0.3904436528682709,  
                             -0.6060700416564941, 0.3892209231853485, -0.15724539756774902, -1.5258700847625732, -0.4823620915412903, -0.008036582730710506, 0.9597200155258179, -0.08309770375490189,
                             0.4133543074131012, 0.10737749189138412, -1.1546218395233154, -0.7463096380233765, 0.6854090094566345, -0.7875686883926392, 0.054122500121593475, -0.4221228063106537,
                             -0.12423055619001389, -1.0514158010482788, 0.49555590748786926, -0.6679829955101013, 0.33741751313209534, 0.08100339025259018, -0.9046081304550171, -0.31811773777008057,
                             -1.0593290328979492, 0.8029569983482361, -0.4746333956718445, 0.2902415990829468, -0.02391364984214306, 0.9472934007644653, -0.6690946817398071, -0.23822452127933502, 0.3979015648365021,
                             0.5280752182006836, 0.48490607738494873, 0.28258585929870605, 0.30200693011283875, -0.3902973234653473, -0.5712165236473083, -0.07573006302118301, 0.062188901007175446,
                             -0.35473892092704773, 0.019009878858923912, -0.3510318994522095, -0.5592910051345825, 0.6185508966445923, 1.3284752368927002, 0.6059293150901794, 0.29538053274154663, -0.299834668636322,
                             -0.005785775370895863, -0.48699817061424255, -0.7243524789810181, 0.5046954154968262, -0.023993313312530518, -0.3906922936439514, -0.6374807953834534, 0.20928744971752167,
                             -0.07345342636108398, -0.7642233371734619, 0.35852470993995667, -0.3917404115200043, -0.24343441426753998, -0.2576097846031189, 0.45362526178359985, -0.12351766228675842,
                             0.04620091989636421, 0.5692318677902222, 0.8238061666488647, 0.05735562741756439, 1.0947062969207764, 0.17684978246688843, -0.21936343610286713, 0.7616904973983765, -0.4831133186817169, 
                             0.13874492049217224, -1.0988963842391968, -0.6850720047950745, -0.524350106716156, -0.2027299553155899, 0.13197481632232666, 0.5966333746910095, -1.7559857368469238,
                             -0.003737833583727479, -0.18901653587818146, -0.3736962080001831, 0.7364284992218018, -0.886589527130127, -0.4643038213253021, -0.0728592574596405, 0.6974549293518066,
                             0.5474512577056885, 18.36789321899414, 0.1313762366771698, 0.849122941493988, -0.057692453265190125, 0.5852394104003906, -0.4771362245082855, -0.3036651015281677, -0.5788114666938782,   
                             -0.32836395502090454, -0.7439397573471069, 0.2054334431886673, 1.055233120918274, 0.6800400018692017, -0.03872578591108322, 0.04516956955194473, 0.7014715075492859, -0.9350305199623108, 
                             0.5017321705818176, -0.13486769795417786, -0.7552860975265503, -0.017354212701320648, -0.3083804249763489, 0.040896251797676086, -0.3213428556919098, -0.6805778741836548,
                             -1.1612604856491089, -0.47846370935440063, 0.8506270051002502, 0.1176915168762207, 0.4423138499259949, -0.11405942589044571, -0.5349186062812805, 0.508454442024231, 0.23639816045761108, 
                             -0.03629712760448456, -0.30732929706573486, 0.8909081816673279, -0.7179247140884399, -0.07150007039308548, 0.5111492276191711, -0.26769402623176575, -1.6655895709991455,
                             -1.1193535327911377, -0.07721060514450073, -0.20205077528953552, -0.9015671014785767, 0.07998819649219513, 0.7257985472679138, 0.3630436360836029, 0.18190544843673706,
                             0.4225357174873352, 0.17464591562747955, -0.43291351199150085, 0.2391851544380188, -0.9908747673034668, 0.6063825488090515, -0.4577066898345947, 0.1189487874507904, 0.030391909182071686,
                             0.4993036687374115, 0.8881727457046509, 0.72125244140625, -0.873413622379303, 0.10198903828859329, 1.2378147840499878, 0.7568597197532654, -0.8926937580108643, -0.2679643929004669,      
                             0.0993872582912445, -0.11188604682683945, 0.5725263357162476, -0.4695080518722534, 0.2065197229385376, -0.03426008298993111, -0.4965148866176605, 0.3644025921821594, 0.5332754254341125, 
                             0.8923283219337463, 0.9350752234458923, 0.33660346269607544, -0.26498448848724365, -0.21952098608016968, 0.13443121314048767, -0.27280473709106445, 0.20703181624412537,
                             -1.1160101890563965, -0.48755568265914917, 0.4937828779220581, -1.0157630443572998, 0.30962538719177246, 0.036219704896211624, -0.09432493150234222, -0.3236379325389862,
                             -0.6408786773681641, 0.15304943919181824, -0.6058834195137024, 0.12257371097803116, -0.16706973314285278, 0.5805904269218445, -0.6235557794570923, -0.7767959833145142,
                             -1.0923115015029907, -0.10280817002058029, 0.21461403369903564, -0.529011070728302, 0.0202870424836874, 0.2652607858181, 0.259946346282959, 0.24872350692749023, 0.057333145290613174,    
                             -0.6980998516082764, -0.38921913504600525, -0.08680473268032074, -1.0609990358352661, 0.3673175871372223, -0.035230062901973724, 0.0028820205479860306, 0.45789414644241333,
                             1.182288408279419, 0.13869240880012512, 0.6094264984130859, 0.1195092424750328, 0.11044542491436005, 0.06419876217842102, -1.2681578397750854, -0.40108269453048706, -0.5761560797691345, 
                             -0.5288388729095459, -0.023758182302117348, 0.506766676902771, 0.029783165082335472, -0.08912313729524612, -0.865385115146637, -0.08268585056066513, 0.624488115310669,
                             0.11585653573274612, 0.5800228714942932, -0.1858023852109909, 0.6412701606750488, 0.11030391603708267, -1.118533968925476, -0.37410956621170044, -0.42297112941741943, 0.8242596387863159,
                             -0.2865186929702759, 0.7050358653068542, -0.34536829590797424, 0.3443280756473541, 0.0961204320192337, -0.3379282057285309, -0.11389844864606857, 0.1977769136428833, -0.6623666286468506,
                             0.021767253056168556, 0.6920328736305237, -0.06543251127004623, -0.3529875576496124, 0.18560436367988586, 1.146639347076416, -0.304966002702713, 0.16120262444019318, 0.22490058839321136,
                             -0.706372857093811, 0.7999510765075684, -0.32337915897369385, 0.4596324563026428, -0.6823104619979858, 1.2726049423217773, 0.20319145917892456, -0.52555251121521, -0.0027076052501797676,
                             0.14053931832313538, -0.14986005425453186, 0.5562940239906311, 1.0327397584915161, 0.6017365455627441, 0.034569598734378815, 0.3188086152076721, -0.24440132081508636, -0.640842080116272,
                             0.3960968852043152, -1.3515413999557495, -0.011669567786157131, -0.9671528339385986, 0.36498889327049255, 0.2446521371603012, -0.0803525373339653, -0.007565246894955635,
                             -0.6041561365127563, 0.04732310771942139, -0.19103528559207916, 0.9045255780220032, 0.7008582949638367, -0.13617821037769318, 0.672210693359375, -0.4473324716091156, -0.7621186971664429,
                             0.04839120805263519, -0.47403961420059204, -0.10752324759960175, 0.7808939218521118, 0.29360726475715637, 0.22485339641571045, -0.7969922423362732, 0.3184205889701843,
                             -0.5338315963745117, -0.5367085933685303, 0.22477005422115326, -0.35338619351387024, -0.12102531641721725, -0.7006676197052002, 0.0941709503531456, 0.3158023953437805, 0.572130024433136,
                             -0.6268612146377563, -1.4823174476623535, 0.622434675693512, -0.07609349489212036, 0.13509200513362885, -0.05670541152358055, 0.3388241231441498, 0.17881324887275696,
                             -0.23474395275115967, -1.0475783348083496, -0.45583927631378174, -0.05653795599937439, 0.44320106506347656, -0.12130533158779144, 0.735500156879425, 0.23705261945724487,
                             0.051757823675870895, -0.6077499389648438, -0.5967379212379456, -1.1217684745788574, -0.05517688766121864, 0.041956827044487, 0.6775431632995605, 0.2591142952442169, -0.2218557894229889,
                             -0.31703993678092957, 0.18158142268657684, -1.4962794780731201, -0.2946910560131073, 1.5616167783737183, -0.5199440121650696, -0.3088962435722351, -0.654837429523468,
                             0.24274301528930664, 0.8711400032043457, 1.0762877464294434, 0.1867835968732834, -0.39209601283073425, -0.6906552314758301, -0.14385153353214264, -0.399618923664093,
                             -0.12058786302804947, 0.4366726279258728, 1.216522216796875, 0.5931786298751831, 0.558085560798645, 0.1561180055141449, -0.1523248702287674, -0.5383061766624451, -0.24317680299282074,   
                             -0.9896393418312073, -0.962105393409729, 0.8659879565238953, -0.45478981733322144, -0.2740103006362915, -0.8923840522766113, -0.5910601019859314, -0.1074865534901619,
                             -0.03546413779258728, -0.47529715299606323, 0.11703982949256897, -0.07271251082420349, 0.16549630463123322, 0.6247514486312866, 0.9230487942695618, 0.13649612665176392,
                             1.0062979459762573, -0.16359862685203552, -0.6400151252746582, -0.5093609690666199, 0.734161913394928, -0.6456604599952698, -0.4494330585002899, 0.12303215265274048, 0.2700439989566803, 
                             0.3734554350376129, -0.9619210958480835, 0.6972970366477966, -1.1533501148223877, -0.8737980723381042, 0.6964439153671265, -0.533576488494873, -0.9449854493141174, 0.9614453315734863,   
                             -0.22351470589637756, 0.20499275624752045, 0.46616455912590027, 0.17054395377635956, -0.2678026556968689, 0.3092465102672577, -0.7955116033554077, -0.0760076642036438,
                             0.38567665219306946, 0.05727911368012428, -0.5220046043395996, -0.9227073788642883, -0.17526431381702423, -1.2932868003845215, 0.08030644059181213, 0.08798868209123611,
                             1.1068239212036133, -0.9220166206359863, 0.3864642381668091, -0.2261999547481537, 0.3596099019050598, 0.43559014797210693, 0.048586949706077576, -0.05581469461321831,
                             -0.8493415713310242, 0.2397119402885437, -0.6649518013000488, 0.7213895916938782, 0.1859794557094574, -0.5864206552505493, -1.2597534656524658, 0.414956659078598, -0.525022029876709,    
                             -0.27329814434051514, -0.5083048343658447, -0.3743606507778168, -0.2834501564502716, -0.2866065502166748, 0.33383122086524963, -0.48792460560798645, 0.48981353640556335,
                             0.10968340188264847, -0.4160130023956299, -0.1565473973751068, 1.3750125169754028, 0.058286089450120926, 0.40094196796417236, -0.2112816870212555, -0.14900949597358704,
                             -0.07536472380161285, -0.0614427775144577, -0.1536576896905899, 0.46926429867744446, -0.7093988060951233, 0.6989726424217224, -1.2431395053863525, -0.08878393471240997,
                             -0.10158617049455643, -1.0512441396713257, 0.0907413512468338, 0.3342107832431793, -0.403260737657547, 1.2375723123550415, 0.3231808543205261, -0.11250878125429153, -0.5664775371551514, 
                             -0.3430079519748688, -0.6954670548439026, -0.5056202411651611, -0.00542836170643568, 0.6416071057319641, -0.23893436789512634, -0.7699282765388489, 0.03296436741948128,
                             -0.22590507566928864, 0.29341042041778564, -0.09827230125665665, 0.0774180144071579, -0.8655024170875549, -0.5924573540687561, 0.3417612910270691, -0.181745707988739, 0.1950760781764984,
                             0.826812207698822, 0.6612370014190674, -0.693805456161499, 0.5213146805763245, 0.8722124695777893, -0.5408694744110107, -0.39087462425231934, 0.42914512753486633, -0.10025554895401001,  
                             -0.4861515462398529, 0.8478769063949585, -0.2591392397880554, -0.2498355507850647, -1.1004141569137573, -0.1759653091430664, -0.5976536273956299, -0.10467401146888733,
                             -0.5571805834770203, -0.5146697163581848, -0.585002601146698, -0.42476317286491394, 0.49136269092559814, -0.27458256483078003, 1.041733741760254, -0.36319687962532043,
                             -0.832115650177002, -0.0018186712404713035, -0.2743959128856659, -0.3371938467025757, -0.5241177082061768, 0.9672160148620605, -1.1789405345916748, 0.909278154373169,
                             0.10705550014972687, 0.4522503912448883, -1.3326839208602905, 0.30238598585128784, -0.13533832132816315, -0.5371308326721191, -0.12662935256958008, 0.14388537406921387,
                             -0.6744322180747986, -0.6165215969085693, -0.2222980558872223, -0.9418571591377258, -0.30476295948028564, 0.41522684693336487, 0.14144845306873322, -0.4270918667316437,
                             0.2855767011642456, 0.06554759293794632, 0.29262539744377136, -0.16480322182178497, -0.11315517127513885, -0.04635429009795189, 0.13619688153266907, -0.8208582997322083,
                             0.3731536567211151, 0.3783283233642578, 0.25469157099723816, -0.9406909942626953, 0.3080059289932251, -0.04487209394574165, 0.08939848095178604, -0.23595742881298065,
                             -0.9996235370635986, 0.8172146677970886, -0.17917689681053162, 0.31093743443489075, 0.4726080000400543, 0.032453157007694244, -0.5107943415641785, 0.3694629669189453, 0.3118513822555542,
                             -0.7780287265777588, -0.2794156074523926, -0.15061192214488983, -0.9421592950820923, -0.23714230954647064, 0.39836129546165466, -0.3668775260448456, 0.24010206758975983,
                             -0.9830848574638367, 0.31478360295295715, 0.46914443373680115, -0.9204843044281006, -0.7094983458518982, 0.35489794611930847, -0.21180570125579834, -0.8877500891685486,
                             0.1175193041563034, -0.45896920561790466, -0.16533301770687103, -0.25079914927482605, -0.007898795418441296, 0.6649718284606934, -0.6266508102416992, 1.0908782482147217,
                             -0.39757099747657776, -0.39053115248680115, -0.3610253632068634, -1.1212595701217651, -0.10147691518068314, 0.1602223515510559, -1.268351435661316, -0.7937559485435486,
                             -0.5991029143333435, -0.6657328605651855, -0.6389527916908264, -0.24522855877876282, 0.4025205373764038, -0.21862831711769104, 0.2752442955970764, -0.9010206460952759,
                             0.03325474262237549, 0.051859479397535324, 0.29868441820144653, -0.7998828291893005, 0.7596865892410278, 0.07494743913412094, -0.1446932703256607, 0.20522408187389374,
                             0.06656603515148163, -0.14925718307495117, 0.970848023891449, 0.2511381506919861, -0.45979470014572144, -1.2890597581863403, 0.5811992883682251, 0.6195440888404846, 1.6261203289031982,  
                             0.22688670456409454, 0.03667793422937393, 0.30131813883781433, -0.44215908646583557, 0.17348846793174744, -1.5477157831192017, -0.9935824871063232, -0.00386343477293849,
                             0.2584696114063263, 1.480439305305481, -0.16919441521167755, -0.8019172549247742, -0.1571538895368576, -0.22986339032649994, 1.228665828704834, 1.2188078165054321, 1.0196527242660522,   
                             0.6584863066673279, 0.1092650443315506, 0.7226707339286804, -0.24074770510196686, 0.7318272590637207, -0.8189460635185242, -0.029902944341301918, 0.07363945245742798,
                             -0.5083490610122681, 0.005681538488715887, -0.5121148228645325, -1.2651993036270142, 0.05831144005060196, -1.0858924388885498, -0.238519549369812, -0.058119192719459534,
                             -0.019850654527544975, 0.08905048668384552, 0.6993280053138733, 0.16899815201759338, 0.11336050927639008, 0.5323147773742676, 0.25681066513061523, 0.38192713260650635,
                             0.9031150937080383, -0.2311399132013321, 0.5930255651473999, -0.11106310039758682, 0.47738251090049744, -0.3988577127456665, 0.8271723985671997, -0.17655447125434875,
                             -0.1132766455411911, -0.29062432050704956, -0.6440953612327576, 0.052234239876270294, -0.23215694725513458, 0.32542407512664795, -0.2780052125453949, -0.6989536285400391,
                             -0.8541481494903564, -0.43125343322753906, 1.2045340538024902, 0.9234006404876709, 0.4102344512939453, 0.06331774592399597, 0.2843780517578125, 0.03712916374206543, -0.2214697301387787, 
                             0.31387031078338623, -0.09328887611627579, 0.5531642436981201, -0.2695474922657013, -0.5712805390357971, -0.6967442035675049, -0.45460811257362366, -0.042688678950071335,
                             0.6113647222518921, -0.018155008554458618, 0.2256559580564499, 0.450716108083725, -0.19663304090499878, -0.06422552466392517, 0.4176441431045532, 0.09774474054574966,
                             -0.5945876240730286, -0.023153839632868767, -0.09871022403240204, 0.30306756496429443, -1.0992945432662964, -0.12679599225521088, 0.2125716358423233, -0.31205958127975464,
                             -0.8564357161521912, -0.44240280985832214, -0.09381408989429474, -0.2763376235961914, -0.6134414672851562, 1.2638661861419678, -0.7284198999404907, 1.516695499420166,
                             0.04025903716683388, -0.05645794793963432, 0.9939822554588318, 0.6668354868888855, 0.28584784269332886, -0.062460508197546005, 0.49796515703201294, -0.4464585781097412,
                             0.8225077986717224, -0.20753343403339386, -0.5807039141654968, -0.3054286539554596, -0.4264286756515503, -0.9609271883964539, -1.2842425107955933, 0.1775459498167038,
                             -0.40338337421417236, 0.7067253589630127, 0.757769763469696, -0.9062315225601196, 0.19069050252437592, -0.3385186791419983, 0.4552976191043854, -0.7011400461196899, 0.6415008306503296,  
                             0.12294299900531769, -0.8445756435394287, -0.33789244294166565
]

In [38]:
# vector search aggreagate pipeline
vectorsearch_pipeline = [

  {
    "$vectorSearch": {
      "exact": True,
      "index": "vector_search_index",
      "limit": 20,
      "path": "embedding_text",
      "queryVector": queryVector
    }
  },
  {
    "$project": {
      "_id": 0,
      "project_id": 1,
      "doc_type" : 1,
      "source_file": 1,
      "text": 1,
      "score": { 
                "$meta": "vectorSearchScore" 
            }
    }
  },
  {
      "$sort": {"score": -1}
  },
  {
      "$limit": 10
  }

]

results = mongo.collection.aggregate(vectorsearch_pipeline)
# print the results
pprint(list(results))

[{'doc_type': 'markdown',
  'project_id': 'PROYECTO_TELLO',
  'score': 0.8302781581878662,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': "2. el concreto tendr una resistencia de f'ci=210kg/cm2 al momento "
          'del tensado\n'
          '3. el slump mximo podr ser superado por medio de la utilizacin de '
          'aditivos plastificantes o superplastificantes. 4. el diseo de '
          'mezcla deber ser tal que se evite la segregacin y el sangrado. **3. '
          'tertiary element: parmetros sismo resistentes norma e.030 (seismic '
          'resistance parameters standard e.030)**\n'
          '- **description**: this section outlines the key seismic design '
          'parameters as per peruvian standard e.030. - **content '
          'breakdown**:\n'
          '- **sis

In [11]:
pipeline = [
    {
        "$match": {
            "doc_type": "image"  # Filtra los documentos donde doc_type sea "image"
        }
    },
    {
        "$project": {  # Opcional: Proyecta solo los campos que necesitas
            "_id": 0,
            "text": 1,
            "project_id": 1,
            "doc_type": 1,
            "source_file": 1
            
        }
    }
]

results = mongo.collection.aggregate(pipeline)
pprint(list(results))

[{'doc_type': 'image',
  'project_id': 'PROYECTO_TELLO',
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio Tello\\VIEW '
                 'SCREENSHOTS\\01\\01_10-01-27-49.png'},
 {'doc_type': 'image',
  'project_id': 'PROYECTO_TELLO',
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio Tello\\VIEW '
                 'SCREENSHOTS\\01\\01_10-01-28-01.png'},
 {'doc_type': 'image',
  'project_id': 'PROYECTO_TELLO',
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio Tello\\VIEW '
                 'SCREENSHOTS\\01\\01_10-01-28-13.png'},
 {'doc_type': 'image',
  'project_id': 'PROYECTO_TELLO',
  'source_file': 'D:\\OneDrive - Instituto 

In [34]:
queryVector = [ 0.0073770200833678246, 0.036874379962682724, 0.013396685011684895, -0.012454125098884106, 0.006854798179119825,
       -0.012422752566635609, -0.010692216455936432, 0.014908943325281143, 0.023162849247455597, -0.021114585921168327,
       -0.03714098408818245, 0.007084718439728022, -5.604219404631294e-05, -0.011259462684392929, -0.004092090297490358,
       0.008079716935753822, 0.005401384551078081, -0.018387315794825554, -0.011013771407306194, -0.009972120635211468,
       -0.008351651951670647, -0.0012140714097768068, -0.00693941256031394, -0.01967424340546131, -0.0031357863917946815,
       0.02719854936003685, -0.003128386801108718, 0.012979879975318909, 0.014838462695479393, 0.016143130138516426,
       -0.011630725115537643, -0.024403996765613556, 0.010855071246623993, -0.053268756717443466, -0.011388879269361496,
       -0.025373997166752815, -0.019300272688269615, 0.016954943537712097, -0.027046535164117813, 0.006700654048472643,
       -0.02133006416261196, -0.011834494769573212, 0.009470326825976372, -0.006841923575848341, 0.00847156997770071,
       -0.023475171998143196, -0.00656708050519228, -0.01571965031325817, 0.02255559340119362, -0.035092685371637344,
       -0.011387652717530727, -0.01354246772825718, 0.03326445072889328, -0.011236858554184437, -0.02820044755935669,
       0.0008699987665750086, 0.017444733530282974, -0.017371829599142075, -0.06098037213087082, 0.015951471403241158,
       -0.002606183523312211, 0.0021910807117819786, -0.032883986830711365, -0.025541631504893303, -0.014450028538703918,
       0.04372810944914818, 0.01436914037913084, -0.009076975286006927, 0.00228316243737936, -0.04096222296357155,
       0.0016145820263773203, 0.010626367293298244, -0.016742059960961342, 0.01166771911084652, -0.06416300684213638,
       -0.009951632469892502, 0.021235745400190353, -0.0016256324015557766, 0.009931265376508236, 0.021205216646194458,
       0.053002383559942245, 0.03372041508555412, 0.015076780691742897, 0.028832122683525085, -0.03808806464076042,
       0.038918931037187576, -0.005443669855594635, -0.016627298668026924, -0.016980944201350212, -0.012386389076709747,
       0.021349558606743813, -0.021491985768079758, -0.02729385904967785, -0.03714758902788162, 0.004279604181647301,
       0.014011605642735958, -0.015385814011096954, -0.02676752582192421, 0.06361781060695648, 0.041625961661338806,
       0.03139316663146019, 0.004855027887970209, -0.029907427728176117, -0.016998959705233574, 0.029189160093665123,
       -0.011440274305641651, 0.03813478723168373, 0.052287615835666656, 0.027605799958109856, 0.01817527413368225,
       0.0031814880203455687, -0.0018146419897675514, -0.0013451172271743417, 0.013954229652881622, 0.010332968086004257,
       -0.015630515292286873, -0.02023698017001152, 0.017090439796447754, 0.0007747467025183141, -0.024346427991986275,
       0.008756170980632305, 0.0038522568065673113, 0.04805558919906616, -0.002325083129107952, -0.010990999639034271,
       -0.007932241074740887, 0.04915070906281471, 0.01425162423402071, -0.028550663962960243, 0.007245668675750494,
       0.0011096361558884382, 0.01644362136721611, -0.058895543217659, 0.00920519232749939, 0.008361485786736012,
       -0.004352723713964224, -0.003055124543607235, 0.005943489260971546, 0.03148472309112549, -0.03903217241168022,
       -0.022418402135372162, 0.01909490115940571, -0.036079779267311096, 0.005417387932538986, 0.0005276743322610855,
       -0.04899019002914429, -0.00343188364058733, -0.0017947717569768429, 0.04277960583567619, -0.0021960781887173653,
       0.021560460329055786, 0.028418714180588722, 0.011089222505688667, -0.00496605085209012, -0.003433701116591692,
       0.03442012518644333, 0.027912244200706482, 0.07897982746362686, -0.015563014894723892, -0.0012225345708429813,
       0.021167173981666565, -0.05881670117378235, -0.041944924741983414, 0.025698337703943253, -0.02317971922457218,
       0.010959791950881481, 0.0628148689866066, 0.005767353344708681, 0.012361239641904831, 0.006283830385655165,
       -0.004189659841358662, -0.0058940802700817585, 0.02061256393790245, -0.022937513887882233, 0.0029856811743229628,
       -0.007616980001330376, 0.01610707677900791, 0.03890810161828995, 0.033963363617658615, -0.048730529844760895,
       -0.02441963367164135, 0.0329865925014019, 0.02671167068183422, 0.011432810686528683, 6.433276575990021e-05,
       0.023936448618769646, 0.030552774667739868, -0.0024286597035825253, 0.01936868205666542, 0.02884177677333355,
       0.01131330244243145, -0.008278992027044296, 0.04208196699619293, -0.011143123731017113, 0.022073382511734962,
       -0.008221559226512909, 0.007714151404798031, 0.01351256761699915, 0.017954034730792046, 0.009462476707994938,
       -0.029427113011479378, -0.014365832321345806, 0.006136809941381216, 0.013659542426466942, -0.03095625527203083,
       0.021649600937962532, -0.05195701867341995, -0.027967415750026703, -0.002249807119369507, -0.050893064588308334,
       0.0014363264199346304, -0.019731372594833374, 0.004820622969418764, 0.015378649346530437, 0.0026211740914732218,
       0.04861344024538994, 0.02399059385061264, -0.03812780603766441, 0.006580481305718422, -0.026449725031852722,
       0.018998850136995316, 0.03561604395508766, -0.006023736204952002, -0.006461023818701506, 0.015535474754869938,
       -0.041010256856679916, 0.024257488548755646, 0.03637963533401489, 0.032519593834877014, -0.03124614991247654,
       -0.029402196407318115, 0.008085762150585651, -0.005698832683265209, -0.01310440432280302, 0.02533789724111557,
       0.007913925684988499, -0.02736000157892704, 0.01736745610833168, 0.04206833988428116, -0.004743256606161594,
       -0.0035552491899579763, -0.022816410288214684, 0.07104209810495377, -0.010031276382505894, -0.0024169590324163437,
       0.029782887548208237, -0.01680518127977848, 0.011243454180657864, 0.01050595659762621, 0.004666908644139767,
       0.005449025891721249, -0.045324504375457764, -0.02337031625211239, -0.0011375872418284416, 0.01862436905503273,
       -0.0004676861281041056, -0.013902894221246243, 0.010282780043780804, 0.022324519231915474, 0.04469100385904312,
       0.010429334826767445, 0.03359916806221008, -0.02182016335427761, 0.005702625494450331, 0.01706238090991974,
       -0.011312968097627163, -0.005948326550424099, -0.011196133680641651, 0.001268729567527771, -0.0004853642894886434,
       -0.006189761217683554, -0.028431931510567665, 0.007652729749679565, -0.006775887217372656, 0.01627209410071373,
       0.020113341510295868, 0.004467587918043137, -0.0017709210515022278, 0.002637416822835803, 0.011490607634186745,
       -0.00775996083393693, -0.0010512055596336722, 0.024004338309168816, -0.04037962108850479, -0.031877487897872925,
       -0.01922549679875374, -0.07144539803266525, -0.002109597437083721, 0.03154638409614563, -0.047989148646593094,
       -0.006261842325329781, 0.0037279881071299314, 0.07422681152820587, -0.00072051165625453, 0.0007047349936328828,
       -0.002960765268653631, -0.04117297753691673, -0.04605285823345184, -0.015212896279990673, -0.01276333350688219,
       0.021446214988827705, 0.014087921939790249, 0.0017962537240236998, -0.02258126065135002, -0.03579274192452431,
       -0.013010616414248943, -0.00801943615078926, -0.00943922158330679, -0.021060312166810036, -0.005677035544067621,
       -0.0034853580873459578, -0.014867667108774185, 0.0065421718172729015, -0.060772042721509933, -0.007319958880543709,
       -0.0038793382700532675, -0.019298817962408066, 0.01797723025083542, -0.04226649925112724, 0.03879228234291077,
       0.011976420879364014, 0.016575653105974197, -0.04061088711023331, 0.02219710312783718, -0.01147956307977438,
       -0.04244842752814293, 0.0005418337532319129, -0.05760924890637398, -0.009417766705155373, -0.010087861679494381,
       0.04047941789031029, -0.0009649450075812638, 0.04221079871058464, 0.007607404142618179, 0.04888572916388512,
       0.0137692391872406, 0.009204883128404617, 0.026073427870869637, 0.013052255846560001, -0.01431199535727501,
       -0.016725892201066017, 0.019684169441461563, -0.011143656447529793, -0.02170027419924736, -0.034928370267152786,
       -0.0034949667751789093, -0.03015335090458393, 0.0334087498486042, -0.039273083209991455, -0.0005715218721888959,
       0.027392685413360596, -0.022990131750702858, 0.01112295687198639, 0.021073026582598686, 0.030542075634002686,
       0.0018094680272042751, -0.021285438910126686, -0.001545487903058529, -0.024271737784147263, 0.027610842138528824,
       0.03658212348818779, 0.01600656285881996, -0.04059087112545967, 0.013905603438615799, -0.007330247666686773,
       0.004566503223031759, -0.028108688071370125, 0.0015223874943330884, 0.004074023105204105, 0.036105163395404816,
       -0.015979105606675148, 0.02435920387506485, 0.027525724843144417, 0.01549905352294445, -0.03806598111987114,
       -0.027676701545715332, -0.07607749849557877, 0.029146356508135796, 0.042155876755714417, -0.057445891201496124,
       0.0038876202888786793, -0.039245568215847015, -0.02222723513841629, -0.002135545015335083, -0.008734790608286858,
       0.026969972997903824, 0.6190284490585327, 0.019832083955407143, 0.04746915400028229, -0.03246842324733734,
       -0.013253364711999893, 0.024333031848073006, 0.032527122646570206, -0.0006469606305472553, -0.005415445659309626,
       -0.04650721698999405, -0.05052223801612854, -0.034717876464128494, 0.03066079132258892, 0.009848522953689098,
       -0.009753863327205181, 0.024725306779146194, -0.006015480030328035, 0.008674995973706245, 0.05563315004110336,
       -0.007795913144946098, 0.009267590008676052, 0.03236939758062363, -0.016099154949188232, 0.009342102333903313,
       0.024845952168107033, -0.02169770561158657, -0.009136429987847805, 0.03145097568631172, 0.013935097493231297,
       0.008016899228096008, 0.010735727846622467, 0.008643711917102337, 0.0260001327842474, -0.028558602556586266,
       -0.015447430312633514, 0.011232452467083931, -0.010341335088014603, -0.016624148935079575, 0.0005254453280940652,
       0.017373835667967796, -0.018583284690976143, -0.03575470298528671, -0.04051636531949043, 0.014175012707710266,
       0.016546208411455154, -0.023164313286542892, -0.012976570054888725, -0.008000660687685013, 0.021418435499072075,
       0.007886981591582298, 0.03229713812470436, 0.03204244375228882, -0.027922797948122025, 0.016871988773345947,
       -0.01886589080095291, -0.008373021148145199, -0.02664625458419323, -0.03667338192462921, 0.018390052020549774,
       0.01751381903886795, 0.009209812618792057, 0.013659171760082245, -0.02728625200688839, -0.025147540494799614,
       -0.02667464129626751, 0.015148130245506763, -0.034882839769124985, -0.0011624537874013186, 0.018624484539031982,
       0.01741347648203373, 0.02338317036628723, -0.008692248724400997, -0.04553265869617462, 0.023760931566357613,
       0.00573863135650754, 0.01387297548353672, -0.0006028530187904835, 0.02434854954481125, -0.03084421530365944,
       0.012520530261099339, -0.006017623003572226, -0.013561046682298183, -0.0024639160837978125, -0.002088929060846567,
       0.023337800055742264, 0.029609058052301407, 0.003937744535505772, -0.005194826051592827, -0.018412841483950615,
       -0.001487631700001657, -0.02351026050746441, -0.006530615035444498, -0.00536552956327796, -0.02274140901863575,
       -0.032563284039497375, 0.013708453625440598, 0.020776472985744476, -0.02872365340590477, -0.038618817925453186,
       0.0005213934346102178, -0.028440047055482864, -0.02952924370765686, 0.01935381256043911, -0.00853747595101595,
       -0.006606319919228554, 0.012119585648179054, 0.02167178876698017, -0.023196153342723846, -0.017932726070284843,
       0.012273149564862251, -0.004112493246793747, 0.02448263391852379, 0.005065527278929949, 0.011999969370663166,
       -0.0020337735768407583, 0.027071513235569, -0.02640008181333542, 0.010678118094801903, 0.027095207944512367,
       0.01870778203010559, -0.030269615352153778, -0.033484622836112976, 0.01857752725481987, -0.03871459513902664,
       0.0016655403887853026, 0.010496888309717178, -0.006226046942174435, -0.0167892724275589, -0.0047883265651762486,
       0.03696770220994949, -0.01099058985710144, -0.021274397149682045, -0.04261058568954468, -0.027913540601730347,
       0.005393066443502903, 6.952276453375816e-05, 0.04163020849227905, 0.006423167418688536, -0.006598321255296469,
       0.0050779785960912704, 0.01879880018532276, -0.035182077437639236, -0.021261140704154968, -0.008842002600431442,
       -0.02286587841808796, -0.013341028243303299, -0.006736173294484615, 0.0012685213005170226, -0.008023128844797611,
       -0.02955475077033043, 0.017845023423433304, -0.017427923157811165, -0.011909152381122112, -0.01756543666124344,
       0.019356610253453255, -0.04527999460697174, -0.013239817693829536, 0.028007909655570984, -0.02495032548904419,
       -0.014685206115245819, -0.0012128580128774047, -2.1935220502200536e-05, 0.01300963293761015, 0.005961576942354441,
       0.02074342966079712, 0.04033679515123367, -0.020944109186530113, 0.03372550383210182, 0.010207954794168472,
       0.02145673707127571, 0.0026200164575129747, -0.00958969071507454, -0.031373701989650726, 0.01531783677637577,
       0.01923559047281742, 0.032211437821388245, -0.02623005025088787, -0.0037199894431978464, -0.030958566814661026,
       0.05887603014707565, 0.055946797132492065, -0.02989129163324833, 0.004898069892078638, -0.023135390132665634,
       0.009336259216070175, 0.014414053410291672, -0.010362125933170319, -0.009427769109606743, 0.021369431167840958,
       -0.017510419711470604, -0.01637633889913559, 0.05510461702942848, -0.006432735361158848, -0.027213459834456444,
       0.03713962435722351, -0.05725427716970444, 0.001024699886329472, -0.020197978243231773, -0.025015955790877342,
       -0.044993095099925995, -0.016209816560149193, -0.0018220373895019293, 0.00357434106990695, 0.007105477619916201,
       -0.03371882066130638, -0.029855581000447273, -0.01704966276884079, -0.011031417176127434, -0.0005983688752166927,
       0.018933508545160294, 0.025514183565974236, -0.021484695374965668, 0.014268958009779453, 0.008998573757708073,
       0.019800512120127678, -0.006614560727030039, 0.03321470692753792, -0.005185561254620552, -0.0012935771374031901,
       -0.021603664383292198, -0.023864323273301125, 0.002552967518568039, 0.04068751260638237, -0.021377217024564743,
       -0.012554377317428589, -0.022032422944903374, 0.01209608931094408, -0.03715331852436066, 0.05415579676628113,
       -0.027307990938425064, -0.011214452795684338, -0.030004529282450676, -0.007774121593683958, -0.007567834109067917,
       0.017989924177527428, 0.0032106346916407347, -0.00874567124992609, -0.001612817868590355, -0.037018004804849625,
       -0.011127550154924393, 0.0035381594207137823, -2.9621085559483618e-05, -0.014243187382817268, 0.018953584134578705,
       -0.02973719872534275, -0.014424039982259274, -0.023914318531751633, 0.01845761388540268, 0.05238651856780052,
       0.006557603366672993, -0.013743016868829727, 0.009263716638088226, 0.007906095124781132, 0.003351852297782898,
       0.0015132767148315907, 0.04047304391860962, 0.011153791099786758, -0.02205573581159115, -0.03341927006840706,
       0.0016032332787290215, 0.022273732349276543, 0.002590677933767438, 0.0033327715937048197, 0.010246755555272102,
       -0.014796514995396137, -0.011989775113761425, 0.04311380162835121, 0.022413838654756546, -0.006904426030814648,
       -0.03824446350336075, -0.03222477436065674, -0.0007904153899289668, -0.023798076435923576, 0.007166392169892788,
       0.03353453427553177, 0.008706063963472843, 0.009554635733366013, -0.0028328094631433487, -0.010919829830527306,
       -0.005516849923878908, -0.01952274702489376, 0.038695525377988815, -0.012719789519906044, -0.0017966150771826506,
       -0.012481068260967731, 0.039239343255758286, 0.026249710470438004, -0.018712345510721207, 0.031168077141046524,
       0.008789832703769207, 0.021457239985466003, -0.0039420840330421925, -0.060974154621362686, 0.04128921777009964,
       -0.018701832741498947, 0.00106350879650563, 0.012992058880627155, 0.010598478838801384, -0.03323148936033249,
       0.038959477096796036, 0.012270343489944935, 0.03675930202007294, -0.02515416592359543, -0.02089734561741352,
       -0.05624110624194145, -0.006692576687783003, -0.03645668551325798, 0.029376769438385963, 0.022848576307296753,
       -0.0015677373157814145, 0.03235439211130142, -0.05432325601577759, 0.008796130307018757, -0.03484196588397026,
       -0.011936855502426624, -0.033163730055093765, 0.008133807219564915, -0.00878081563860178, -0.02309398725628853,
       0.018637513741850853, -0.0006886704941280186, 0.0005632500979118049, 0.002078230958431959, 0.03231034427881241,
       -0.02185698412358761, -0.0011551548959687352, -0.03194570168852806, -0.03785979747772217, -0.0034089735709130764,
       -0.03974117338657379, 0.008684376254677773, -0.01285774726420641, -0.021284451708197594, -0.012881545349955559,
       0.017105214297771454, 0.03906696289777756, -0.017824307084083557, -0.02831663191318512, 0.02171793393790722,
       0.002990966197103262, -0.0105860261246562, -0.020852310582995415, 0.03243561089038849, -0.003995505161583424,
       0.0010070479474961758, 0.009321014396846294, -0.010851996019482613, 0.007948387414216995, -0.00628559710457921,
       -0.00012384966248646379, 0.04109306260943413, 0.05095645785331726, 0.015946675091981888, -0.05541270226240158,
       -0.010596105828881264, -0.01290408056229353, 0.009218205697834492, 0.012255064211785793, 0.020967984572052956,
       -0.02937844954431057, -0.008147093467414379, 0.0005201225867494941, -0.013533144257962704, 0.01298697292804718,
       -0.02010113000869751, -0.01145895104855299, 0.00029730977257713675, -0.005461100023239851, -5.8731438912218437e-05,
       0.014742936939001083, 0.015774907544255257, 0.04619206488132477, 0.03215634450316429, -0.02014239877462387,
       -0.036135558038949966, 0.043135445564985275, 0.0066353087313473225, -0.014263777062296867, 0.02315112203359604,
       0.014158724807202816, 0.009406540542840958, -0.025699879974126816, -0.023349052295088768, -0.017480384558439255,
       0.016070494428277016, -0.013577166944742203, -0.01854459010064602, 0.01401379145681858, 0.027788758277893066,
       0.042213208973407745, -0.021374886855483055, 0.02562841959297657, -0.038768116384744644, 0.030979374423623085,
       -0.08905436843633652, 0.0016377813881263137, -0.01632045954465866, -0.034473445266485214, -0.018649721518158913,
       -0.009415628388524055, -0.045429013669490814, -0.004766583908349276, -0.03643329441547394, -0.004959541838616133,
       -0.0026864237152040005, -0.014442789368331432, -0.02128174528479576, -0.019416838884353638, -0.006504513323307037,
       0.0003142074856441468, -0.017840301617980003, -0.0027728616259992123, -0.01344482135027647, 0.020776599645614624,
       -0.012154372408986092, -0.006871733348816633, -0.028613314032554626, -0.005687248427420855, -0.011958664283156395,
       0.02236342616379261, 0.01812414452433586, -0.003051335457712412, -0.03940342739224434, 0.009156868793070316,
       0.06972654163837433, 0.001260611112229526, -0.022314313799142838, 0.0350419245660305, 0.01543815340846777,
       -0.006729643791913986, -0.005767878144979477, -0.016574854031205177, 0.01485133171081543, -0.005812553688883781,
       0.028141111135482788, -0.0004883451038040221, 0.013508373871445656, -0.018909545615315437, -0.012106797657907009,
       0.06443122029304504, -0.02477743662893772, -0.027140669524669647, -0.054114293307065964, -0.029029525816440582,
       -0.030077369883656502, 0.03295094147324562, 0.03167635574936867, 0.0112924138084054, -0.020058684051036835,
       -0.02131226845085621, -0.03170002996921539, -0.0029302453622221947, -0.004985453095287085, -0.00598090561106801,
       -0.037531886249780655, 0.017704689875245094, 0.016740858554840088, -0.059056855738162994, -0.033412814140319824,
       0.005452550482004881, -0.030099695548415184, -0.01388782449066639, 9.368987230118364e-05, -0.06271325796842575,
       -0.005739599000662565, 0.0402778685092926, 0.022501083090901375, -0.012373012490570545, -0.0017944752471521497,
       0.015826180577278137, -0.02354160137474537, -0.004298579413443804, 0.0019591841846704483, -0.022553961724042892,
       -0.018102344125509262, 0.007287546060979366, 0.006285764742642641, -0.029121093451976776, 0.013960159383714199,
       0.010575526393949986, -0.007692802231758833, -0.0018608502577990294, -0.02004092186689377, 0.014357226900756359,
       -0.03321799635887146, -0.008112392388284206, 0.017785821110010147, -0.008739260956645012, 0.018957743421196938,
       -0.04581626132130623, -0.01604481227695942, 0.04777337610721588, -0.013542035594582558, 0.0007914358866401017,
       0.012146279215812683, 0.003382744500413537, 0.03578402101993561, -0.041710302233695984, -0.014275917783379555,
       -0.0074826497584581375, 0.020715711638331413, 0.03876533731818199, 0.010922829620540142, -0.0035603668075054884,
       -0.04841940477490425, -0.01975850760936737, -0.05677558109164238, 0.009316731244325638, -0.020893825218081474,
       0.023993566632270813, -0.00934290885925293, 0.011833404190838337, -0.02730751596391201, -0.024558795616030693,
       0.007542770821601152, 0.0015776879154145718, 0.028709251433610916, -0.02665097452700138, -0.03271682187914848,
       -0.011598263867199421, 0.011708158068358898, 0.0138748399913311, -0.011610605753958225, -0.002472763182595372,
       0.029140925034880638, -0.03403567895293236, 0.02109442837536335, 0.020834505558013916, 0.019172685220837593,
       0.03767929971218109, -0.031055836006999016, 0.025454310700297356, -0.012654267251491547, -0.021021386608481407,
       0.03267742320895195, 0.012870592065155506, 0.0151819521561265, -0.020203419029712677, -0.022294575348496437,
       -0.030696138739585876, 0.019527632743120193, -0.027948835864663124, -0.01357924472540617, -0.020252620801329613,
       -0.011862661689519882, 0.013491126708686352, 0.010769980028271675, 0.015170671977102757, 0.0005324477097019553,
       -0.02921890839934349, 0.013111129403114319, -0.06530437618494034, 0.02758122608065605, 8.674380660522729e-05,
       0.032803405076265335, -0.0541386604309082, -0.01133730635046959, -0.016567474231123924, -0.01069725677371025,
       0.05706460028886795, 0.02065063640475273, -0.004076237790286541, -0.02844015508890152, -0.01876099780201912,
       -0.021694323047995567, 0.06358906626701355, -0.014220820739865303, 0.018713703379034996, -0.00298955081962049,
       -0.01099238358438015, 0.016672397032380104, -0.010235724970698357, 0.039602916687726974, -0.017461637035012245,
       0.029816506430506706, 0.03684898465871811, -0.0036271833814680576, 0.0007095555192790926, -0.006233721040189266,
       0.03917865827679634, 0.020859118551015854, -0.01239506434649229, -0.031742025166749954, -0.01783706434071064,
       -0.025805888697504997, -0.007795784156769514, -0.001948820659890771, -0.014371437951922417, 0.006062368396669626,
       0.0015061595477163792, 0.01075308583676815, 0.016510868445038795, -0.013342468999326229, -0.006729756016284227,
       -0.024413105100393295, -0.012945190072059631, -0.0177227184176445, -0.00681327236816287, 0.0031179962679743767,
       -0.029541805386543274, -0.0043716817162930965, -0.05714431405067444, -0.02485213801264763, -0.02450493909418583,
       -0.016833076253533363, -0.008301304653286934, -0.03977913409471512, 0.005019575357437134, -0.02581675350666046,
       -0.004683798179030418, 0.030516549944877625, 4.516078479355201e-05, 0.001866639475338161, -0.020226499065756798,
       0.0011122970608994365, 0.021434564143419266, 0.021719809621572495, -0.0030803647823631763, -0.022912366315722466,
       -0.05876874178647995, 0.002045473549515009, -0.021184314042329788, 0.005259573459625244, -0.021253511309623718,
       -0.051662325859069824, -0.017889222130179405, -0.01719590276479721, 0.046808887273073196, 0.009279360063374043,
       0.013341302052140236, -0.00675165606662631, 7.33009583200328e-05, 0.03258690610527992, -0.010682537220418453,
       0.04191944748163223, 0.010431542992591858, -0.05731044337153435, 0.042697831988334656
]

In [80]:
pipeline = [
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 400,
            "limit": 100,
            "path": "embedding_text",
            "queryVector": queryVector
        }
    },
    {
        "$project": {
            "_id": 0,
            "project_id": 1,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    {
        "$match": {
            "score": { "$gt": 0.2 }
        }
    },
    {
        "$setWindowFields": {  # Ordena y asigna un ranking dentro de cada grupo
            "partitionBy": "$doc_type",
            "sortBy": { "score": -1 },
            "output": {
                "rank": { "$rank": {} }
            }
        }
    },
    {
        "$match": {
            "rank": { "$lte": 10 }  # Filtra los 3 mejores documentos por grupo
        }
    },
    {
        "$project": {  # Opcional: Limita los campos en los resultados finales
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": 1
        }
    }
    
    
]

results = mongo.collection.aggregate(pipeline)
pprint(list(results))

[{'doc_type': 'markdown',
  'score': 0.8302781581878662,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': "2. el concreto tendr una resistencia de f'ci=210kg/cm2 al momento "
          'del tensado\n'
          '3. el slump mximo podr ser superado por medio de la utilizacin de '
          'aditivos plastificantes o superplastificantes. 4. el diseo de '
          'mezcla deber ser tal que se evite la segregacin y el sangrado. **3. '
          'tertiary element: parmetros sismo resistentes norma e.030 (seismic '
          'resistance parameters standard e.030)**\n'
          '- **description**: this section outlines the key seismic design '
          'parameters as per peruvian standard e.030. - **content '
          'breakdown**:\n'
          '- **sistema estructural sismorresistente*

In [35]:
# ...existing code...
from pymongo.operations import SearchIndexModel

# Nuevo índice con campos de filtro
search_index_model = SearchIndexModel(
    definition={
        "fields": [
            {
                "type": "vector",
                "path": "embedding_text",
                "numDimensions": 1024,
                "similarity": "cosine"
            },
            { "type": "filter", "path": "doc_type" },
            { "type": "filter", "path": "project_id" },      # opcional
            { "type": "filter", "path": "source_file" }      # opcional
        ]
    },
    name="vector_search_index",
    type="vectorSearch",
)

mongo.collection.create_search_index(search_index_model)

# (Opcional) listar para verificar
for idx in mongo.collection.list_search_indexes():
    print(idx.get("name"))
# ...existing code...

vector_text
vector_search_index


In [36]:
for idx in mongo.collection.list_search_indexes():
    if idx.get("name") == "vector_search_index":
        print(idx.get("name"), "| status:", idx.get("status"), "| fields:",
              [f.get("path") for f in idx.get("latestDefinition", {}).get("fields", [])])

vector_search_index | status: PENDING | fields: ['embedding_text', 'doc_type', 'project_id', 'source_file']


In [9]:
mongo.db.command("dropSearchIndex", mongo.collection.name, name="vector_search_index")

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1759373282, 1),
  'signature': {'hash': b'j\x16\x9b\xf8\x1d\xda\x8b\x88\x97\xb9\x19\x82p\xcaA\x1a\x8c\xfc\xdc\xf6',
   'keyId': 7526364577701822470}},
 'operationTime': Timestamp(1759373282, 1)}

In [34]:
for idx in mongo.collection.list_search_indexes():
    print(idx.get("name"))

vector_text


In [96]:
# ...existing code...
pipeline_markdown_only = [
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 200,
            "limit": 50,
            "path": "embedding_text",
            "queryVector": queryVector,
            "filter": { "doc_type": "markdown" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    { "$sort": { "score": -1 } },
    { "$limit": 3 }
]

pprint(list(mongo.collection.aggregate(pipeline_image_only)))
# ...existing code...

[{'doc_type': 'markdown',
  'score': 0.8302781581878662,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': "2. el concreto tendr una resistencia de f'ci=210kg/cm2 al momento "
          'del tensado\n'
          '3. el slump mximo podr ser superado por medio de la utilizacin de '
          'aditivos plastificantes o superplastificantes. 4. el diseo de '
          'mezcla deber ser tal que se evite la segregacin y el sangrado. **3. '
          'tertiary element: parmetros sismo resistentes norma e.030 (seismic '
          'resistance parameters standard e.030)**\n'
          '- **description**: this section outlines the key seismic design '
          'parameters as per peruvian standard e.030. - **content '
          'breakdown**:\n'
          '- **sistema estructural sismorresistente*

In [24]:
# ...existing code...
pipeline_image_only = [
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 200,
            "limit": 50,
            "path": "embedding_text",
            "queryVector": queryVector,
            "filter": { "doc_type": "image" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    { "$sort": { "score": -1 } },
    { "$limit": 3 }
]

pprint(list(mongo.collection.aggregate(pipeline_image_only)))
# ...existing code...

[]


In [23]:
# ...existing code...
dist_pipeline = [
  {
    "$vectorSearch": {
      "exact": True,
      "index": "vector_search_index",
      "candidates": 200,
      "limit": 200,
      "path": "embedding_text",
      "queryVector": queryVector
    }
  },
  {"$group": {"_id": "$doc_type", "count": {"$sum": 1}}},
  {"$sort": {"count": -1}}
]
pprint(list(mongo.collection.aggregate(dist_pipeline)))

[]


In [67]:
# Top-N balanceado por doc_type
pipeline_balanced = [
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 400,   # subir para dar espacio a que entren imágenes
            "limit": 400,
            "path": "embedding_text",
            "queryVector": queryVector
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    {
        "$setWindowFields": {
            "partitionBy": "$doc_type",
            "sortBy": { "score": -1 },
            "output": { "rank": { "$rank": {} } }
        }
    },
    { "$match": { "rank": { "$lte": 3 } } },  # top 10 por tipo
    { "$sort": { "score": -1 } }
]

pprint(list(mongo.collection.aggregate(pipeline_balanced)))

[{'doc_type': 'markdown',
  'rank': 1,
  'score': 0.8302781581878662,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': "2. el concreto tendr una resistencia de f'ci=210kg/cm2 al momento "
          'del tensado\n'
          '3. el slump mximo podr ser superado por medio de la utilizacin de '
          'aditivos plastificantes o superplastificantes. 4. el diseo de '
          'mezcla deber ser tal que se evite la segregacin y el sangrado. **3. '
          'tertiary element: parmetros sismo resistentes norma e.030 (seismic '
          'resistance parameters standard e.030)**\n'
          '- **description**: this section outlines the key seismic design '
          'parameters as per peruvian standard e.030. - **content '
          'breakdown**:\n'
          '- **sistema estructural sism

In [86]:
# ...existing code...
pipeline_balanced = [
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 400,
            "limit": 400,
            "path": "embedding_text",
            "queryVector": queryVector
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    {
        "$setWindowFields": {
            "partitionBy": "$doc_type",
            "sortBy": { "score": -1 },
            "output": {
                "rank": { "$rank": {} }
            }
        }
    },
    # Top 5 por doc_type (cambia a 3 si quieres top 3)
    { "$match": { "rank": { "$lte": 5 } } },
    # Agrupamos por doc_type y conservamos los documentos ya filtrados
    {
        "$group": {
            "_id": "$doc_type",
            "docs": {
                "$push": {
                    "source_file": "$source_file",
                    "text": "$text",
                    "score": "$score"
                }
            }
        }
    },
    # (Opcional) asegurar máximo 5 (o 3) por si cambiaste el $match
    {
        "$project": {
            "_id": 0,
            "doc_type": "$_id",
            "docs": { "$slice": ["$docs", 3] }  # cambia 5 a 3 si usas top 3
        }
    },
    { "$sort": { "doc_type": 1 } }
]

pprint(list(mongo.collection.aggregate(pipeline_balanced)))
# ...existing code...

[{'doc_type': 'image',
  'docs': [{'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\26\\26_09-22-13-44.png'},
           {'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\24\\11_09-21-02-40.png'},
           {'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\23\\23_09-20-42-47.png'}]},
 {'

In [93]:
pipeline_unificado = [
    # 1. Ejecutar el primer vectorSearch (para 'markdown')
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 200,
            "limit": 50, # Límite intermedio para esta búsqueda
            "path": "embedding_text",
            "queryVector": queryVector,
            "filter": { "doc_type": "markdown" }
        }
    },
    # 2. Proyectar y obtener el score para 'markdown'
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    # 3. Limitar a 3 mejores resultados de 'markdown' (como en tu pipeline original)
    { "$sort": { "score": -1 } },
    { "$limit": 3 },
    
    # 4. Usar $unionWith para ejecutar la segunda búsqueda (para 'image')
    {
        "$unionWith": {
            "coll": "multimodal-rag", # ¡IMPORTANTE! Reemplaza con el nombre de tu colección
            "pipeline": [
                {
                    "$vectorSearch": {
                        "exact": True,
                        "index": "vector_search_index",
                        "candidates": 200,
                        "limit": 50, # Límite intermedio para esta búsqueda
                        "path": "embedding_text",
                        "queryVector": queryVector,
                        "filter": { "doc_type": "image" }
                    }
                },
                # Proyectar y obtener el score para 'image'
                {
                    "$project": {
                        "_id": 0,
                        "doc_type": 1,
                        "source_file": 1,
                        "text": 1,
                        "score": { "$meta": "vectorSearchScore" }
                    }
                },
                # Limitar a 3 mejores resultados de 'image' (como en tu pipeline original)
                { "$sort": { "score": -1 } },
                { "$limit": 3 }
            ]
        }
    }
    # NOTA: No se requiere un sort/limit final a menos que quieras reordenar todo.
    # Esta estructura garantiza 3 resultados de markdown y 3 de image, si están disponibles.
]

pprint(list(mongo.collection.aggregate(pipeline_balanced)))

[{'doc_type': 'image',
  'docs': [{'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\26\\26_09-22-13-44.png'},
           {'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\24\\11_09-21-02-40.png'},
           {'score': 0.7720474004745483,
            'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                           'Superiores de '
                           'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                           'Tello\\VIEW SCREENSHOTS\\23\\23_09-20-42-47.png'}]},
 {'

In [22]:
pipeline_plano_individual = [
    # 1. Búsqueda de 'markdown'
    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 200,
            "limit": 50,
            "path": "embedding_text",
            "queryVector": queryVector,
            "filter": { "doc_type": "markdown" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    { "$sort": { "score": -1 } },
    { "$limit": 3 }, # Top 3 markdown
    
    # 2. Unión con la búsqueda de 'image' (SIN AGRUPAR)
    {
        "$unionWith": {
            "coll": "multimodal-rag", # Tu colección
            "pipeline": [
                {
                    "$vectorSearch": {
                        "exact": True,
                        "index": "vector_search_index",
                        "candidates": 200,
                        "limit": 50,
                        "path": "embedding_text",
                        "queryVector": queryVector,
                        "filter": { "doc_type": "image" }
                    }
                },
                {
                    "$project": {
                        "_id": 0,
                        "doc_type": 1,
                        "source_file": 1,
                        "text": 1,
                        "score": { "$meta": "vectorSearchScore" }
                    }
                },
                { "$sort": { "score": -1 } },
                { "$limit": 3 } # Top 3 image
            ]
        }
    }
    # NOTA: No hay etapas adicionales de $group o $facet aquí.
]
pprint(list(mongo.collection.aggregate(pipeline_plano_individual)))

[]


In [2]:
queryVector = [-0.06239064782857895,
                             0.6715010404586792, 0.12546344101428986, -0.3955218195915222, -0.4466532766819, -0.6054791808128357, -0.09149301052093506, 1.2518073320388794,
                             -0.7905671000480652, -0.1376897096633911, -0.5017892122268677, 0.7530708909034729, -0.45059704780578613, -0.17407020926475525, 0.18029236793518066,
                             1.0441802740097046, -0.4489714205265045, -0.8537845611572266, 0.3958132266998291, -0.22686424851417542, -0.3809553384780884, -0.5225611329078674,
                             -0.19930097460746765, -0.5667767524719238, 0.5739073753356934, 0.6497673988342285, -0.71403568983078, -0.2932020425796509, -0.5064831376075745,
                             0.5260035395622253, 0.3980380892753601, 0.2572784423828125, 1.0012741088867188, -0.6205945014953613, 0.11821546405553818, -0.5523533225059509,
                             0.47737306356430054, -0.3851184844970703, -0.4883196949958801, 0.34883445501327515, -0.479513019323349, 0.42907002568244934, 0.557018518447876,
                             -0.0962359607219696, -0.2790548801422119, -0.3475663661956787, -0.8794019222259521, -0.6572843194007874, 1.4011400938034058, -0.9718787670135498,
                             -0.49328580498695374, -0.841890811920166, -0.17657756805419922, -0.4434100091457367, -0.07278566807508469, -0.4148220121860504, -0.062448762357234955,        
                             -0.9286829233169556, -1.1399383544921875, 0.09803593158721924, -1.1887285709381104, 0.7565784454345703, -1.2119276523590088, -0.18285855650901794,
                             -0.5800747871398926, 1.0110690593719482, 0.6327665448188782, -0.8076989054679871, 0.042376741766929626, -1.2063367366790771, -0.020853599533438683,
                             0.24067239463329315, 0.1475628763437271, 0.10662981867790222, -0.7934830188751221, 0.35070115327835083, 0.6262584924697876, 0.44020870327949524,
                             0.6680855751037598, 0.1117723137140274, 1.0035090446472168, 0.671649158000946, 0.6974451541900635, 0.2839629054069519, -1.406229019165039, 0.8202403783798218,
                             0.052754517644643784, -0.4584597647190094, -0.5491533875465393, -0.9718209505081177, -0.4627561569213867, -0.06033621355891228, -0.386888325214386,
                             -0.43690016865730286, 0.5233033299446106, -0.9891469478607178, -1.4044113159179688, -0.4904070794582367, 0.3836122751235962, 0.5620434284210205,
                             0.6227825284004211, 0.9221557974815369, -0.11711438000202179, 0.24887561798095703, 0.18585613369941711, -1.2073330879211426, 0.48204049468040466,
                             -0.33818763494491577, 0.3744131326675415, 0.20792213082313538, -0.15663191676139832, 0.9277266263961792, 0.606497049331665, -0.1783023476600647,
                             -0.1121419370174408, -1.001338243484497, -0.7405074238777161, 0.011066390201449394, 0.15178276598453522, -0.43260613083839417, 0.14190837740898132,
                             1.1412529945373535, 0.3501012623310089, -0.8442509770393372, 0.40741652250289917, -0.37648606300354004, 0.3089863061904907, 1.172134280204773,
                             0.06312524527311325, 0.7102234959602356, -0.3211213946342468, 1.2482696771621704, -1.058562994003296, 0.05611260607838631, 0.2475113868713379,
                             -0.19816461205482483, -0.47226205468177795, 0.7361932992935181, 0.5108594298362732, -1.73502516746521, 0.5095235705375671, 0.4284447133541107,
                             -0.32213497161865234, 0.48523589968681335, 0.36035972833633423, -1.008577585220337, 0.20363542437553406, -0.6735400557518005, 1.2201749086380005,
                             0.18023993074893951, 0.41662588715553284, 1.5067956447601318, -0.22776243090629578, -0.1302112638950348, 0.4065057337284088, 0.011667684651911259,
                             0.568507194519043, 0.5750805139541626, 0.0322393961250782, 0.2858900725841522, 0.4103957414627075, -0.36913979053497314, 0.020768556743860245,
                             0.052294496446847916, 0.23433437943458557, 0.1547866016626358, 0.46754175424575806, 1.5490660667419434, 0.08691442012786865, 0.23929408192634583,
                             0.21759021282196045, 0.2112598717212677, -0.4402564764022827, -0.13924860954284668, 0.7350049614906311, -0.06385711580514908, 0.57441246509552,
                             0.8251633644104004, 0.03991462290287018, -0.4023246169090271, 0.17322422564029694, 0.7794906497001648, 0.5326796770095825, -0.2935175597667694,
                             -0.3836451470851898, 0.5927574038505554, 0.47764045000076294, 0.17428769171237946, 0.2279660999774933, 0.13690070807933807, 0.5031429529190063,
                             -0.03146229684352875, 0.3406127393245697, -0.336350679397583, 1.0451018810272217, 0.46698540449142456, -0.35273993015289307, 0.5108062624931335,
                             0.2189803570508957, -0.30237358808517456, 0.01016727089881897, -0.9404788613319397, -0.07043682783842087, -0.011517949402332306, -0.525331974029541,
                             0.9387795925140381, -1.5212770700454712, 0.06925810873508453, -0.6685715317726135, -0.49000510573387146, -0.7865372896194458, 0.7274129986763,
                             -0.5831141471862793, 1.0862538814544678, -0.607101559638977, 0.5278230905532837, 0.44533929228782654, -1.256792426109314, 0.16531530022621155,
                             -0.2391176074743271, 0.04300441965460777, 0.22831431031227112, -0.3345213234424591, -0.5139886736869812, -0.25761136412620544, -1.2009994983673096,
                             0.2975137233734131, 0.27702492475509644, -0.048609714955091476, -1.5795514583587646, -0.3236691653728485, 0.46861305832862854, 0.0843672901391983,
                             -0.9629080891609192, 1.3336079120635986, 0.6457779407501221, 0.07438253611326218, 0.07436878979206085, 0.36299628019332886, -0.3485163152217865,
                             -0.9271044731140137, -0.6418159008026123, 0.6606990694999695, 0.04949098825454712, -0.6908460855484009, 0.5686320066452026, 0.13102859258651733,
                             0.7550315260887146, -0.35751011967658997, -0.22175538539886475, 0.19699189066886902, -1.0185133218765259, -0.23162011802196503, -0.3187999725341797,
                             -0.655760645866394, -0.13428866863250732, -0.43167969584465027, -0.39060303568840027, 0.0853753536939621, 0.44646918773651123, 0.5388504266738892,
                             1.0521575212478638, -0.5266053676605225, -0.7633336782455444, 0.012606155127286911, 0.8300617933273315, -0.03551563620567322, -1.0171713829040527,
                             0.19845718145370483, -0.8166816234588623, -0.5569892525672913, -0.08550834655761719, 0.17202655971050262, -0.8392753601074219, -0.3669571280479431,
                             0.10082809627056122, -0.1072288230061531, 0.42525190114974976, 0.9442338943481445, -0.31101900339126587, -0.4507402181625366, 0.21165290474891663,
                             -0.166630819439888, -0.37436652183532715, -0.4864252805709839, -0.6912171244621277, -0.7734353542327881, -0.34883034229278564, 1.523942232131958,
                             -0.6425374746322632, 0.17413966357707977, 0.32617703080177307, 2.2522494792938232, 0.35786256194114685, -0.08110503107309341, 0.12196620553731918,
                             -0.927664041519165, -2.0234692096710205, -0.895154595375061, 0.18242394924163818, 0.27209052443504333, 0.5976297855377197, 0.48692458868026733,
                             0.47646424174308777, 0.07543027400970459, -0.8520456552505493, -0.5053239464759827, -1.4654982089996338, -0.691173791885376, -0.20335888862609863,
                             -0.3907729685306549, -0.04247415065765381, -0.32613101601600647, -0.18089008331298828, 1.0324188470840454, -0.4117528200149536, -0.3534696698188782,
                             0.8316564559936523, 0.19368934631347656, 1.1367331743240356, 0.44074201583862305, 1.0672991275787354, 0.20689499378204346, 0.8628891706466675,
                             0.37673458456993103, 0.017663370817899704, -0.343436598777771, -1.2970640659332275, -0.35566771030426025, -0.18791066110134125, 0.8929320573806763,
                             0.7156105041503906, 0.8165913820266724, 0.20191866159439087, -7.164850831031799e-05, -0.2366105020046234, -0.36525702476501465, 0.675875723361969,
                             0.6136681437492371, -0.18816962838172913, -0.3987201750278473, 0.07258167117834091, -0.233894944190979, -0.29252415895462036, -0.3253787159919739,
                             -0.685337483882904, 0.17049755156040192, -0.03383931145071983, -0.299456924200058, 0.7179810404777527, 0.9561532735824585, -0.6614269018173218,
                             -0.42969292402267456, 1.4008574485778809, 1.03949773311615, 0.6802300214767456, -0.10302089899778366, -0.1929035484790802, -0.8123027086257935,
                             0.11335593461990356, 0.5456793904304504, -0.2817450761795044, -0.5953996777534485, 0.6454647779464722, -0.44965267181396484, -0.0008675754070281982,
                             -0.8802412748336792, -0.049567319452762604, -0.20007538795471191, -0.4560772776603699, -1.7968045473098755, -0.26610779762268066, -0.14079150557518005,       
                             0.20376348495483398, -0.7243033051490784, -1.0221558809280396, -2.4707841873168945, -0.20312908291816711, 0.8381606340408325, -0.7426486015319824,
                             0.6368182301521301, -1.1454613208770752, -0.4067113697528839, -0.3726082146167755, 0.48861244320869446, 0.6411100029945374, 17.589679718017578,
                             0.31563037633895874, 0.10694865137338638, -0.516482412815094, 0.36119768023490906, -0.042151086032390594, 1.2182022333145142, -0.49050381779670715,
                             0.7488835453987122, -0.6813970804214478, -0.5385476350784302, -0.6452445983886719, -0.27408459782600403, -0.4610726237297058, 0.6609355211257935,
                             0.2020760476589203, -0.03316792845726013, -0.27851611375808716, 1.2247967720031738, -0.11209073662757874, 0.5116584897041321, 0.2656942903995514,
                             -0.1274021863937378, -0.05700068175792694, -0.5457478165626526, -0.8376337289810181, 0.3300616443157196, 0.624811589717865, -0.565491259098053,
                             1.3533947467803955, -0.17198312282562256, 0.2194884717464447, -0.08891350030899048, -0.15770040452480316, 0.009754396975040436, -0.24497349560260773,
                             -0.8121529221534729, -0.7463792562484741, -0.28031620383262634, 0.7523297071456909, -0.3267671763896942, -0.5308290719985962, -1.315535545349121,
                             0.710595428943634, 0.22378423810005188, -0.25842347741127014, -0.016274066641926765, -0.4402948021888733, -0.23747380077838898, 0.6899163126945496,
                             1.1337662935256958, 0.594427227973938, -0.7529275417327881, 0.8088274598121643, -0.06735369563102722, 0.2542032301425934, 0.0340641587972641,
                             -0.6385214328765869, 0.41781362891197205, 0.9382486343383789, -0.04466712102293968, -0.14031395316123962, -0.6432667970657349, -0.11531820893287659,
                             -0.7812743186950684, -0.2058461308479309, -1.2470574378967285, -0.31667786836624146, -0.4349350929260254, 0.6687545776367188, 0.4952859580516815,
                             0.5111085176467896, -0.6032345294952393, -0.28245848417282104, 1.2332993745803833, -0.781558096408844, -0.12378132343292236, 1.8438997268676758,
                             -0.34066975116729736, 0.7580974698066711, 0.7661924362182617, -1.1378023624420166, -0.8770946860313416, -0.7840266823768616, 0.8121426701545715,
                             0.12227903306484222, -0.11122037470340729, -0.250382661819458, -0.19959895312786102, 0.2207418531179428, -0.2821233570575714, -0.3102158308029175,
                             0.17820338904857635, 0.1462329477071762, 1.2093055248260498, 0.3869542181491852, 0.0562269352376461, 0.20297497510910034, -0.8889407515525818,
                             -0.6465891599655151, -1.1003735065460205, 0.21667416393756866, 0.24746042490005493, -0.2325049191713333, -0.17768831551074982, -0.4944121241569519,
                             0.26393622159957886, -0.5638546943664551, -0.4680415093898773, -0.3310225307941437, -0.31898775696754456, 0.4579232335090637, -0.7294041514396667,
                             0.16209609806537628, -0.2047676146030426, -0.265664279460907, -0.8940625190734863, -0.42100805044174194, 0.5222998261451721, -0.4953654110431671,
                             -0.8956804871559143, -0.12483564019203186, 0.4943702220916748, -0.4674433469772339, -0.5948563814163208, -0.5198882818222046, -0.35402733087539673,
                             -0.11711278557777405, -0.4219883978366852, -0.2138611376285553, -0.21221165359020233, -0.030789073556661606, -0.7874507308006287, -0.8843367099761963,        
                             0.7502396106719971, 0.08824145793914795, 0.6658163666725159, -0.426616370677948, 0.804347038269043, 0.33899492025375366, -0.5691953301429749,
                             -1.1664241552352905, -1.0329257249832153, -0.006653085350990295, -0.7172170281410217, -0.11400939524173737, 0.043102700263261795, -0.5030766129493713,        
                             -0.7513366341590881, -0.6675252318382263, 0.41317877173423767, -0.04580641910433769, 0.0005057491362094879, -1.5777199268341064, 0.20158477127552032,
                             -0.05817347764968872, 0.055782221257686615, 0.8799166083335876, -0.2031002640724182, -0.4446031153202057, 0.08655226975679398, 0.3422158360481262,
                             1.0276436805725098, 0.7276604175567627, 0.6330407857894897, 0.2064991593360901, -0.642957866191864, 0.9121994376182556, 0.38863933086395264,
                             -0.040024787187576294, -0.0851885974407196, 0.4247404634952545, -0.995505154132843, 0.7329323887825012, 0.3629692792892456, 0.6469846367835999,
                             -0.4230318069458008, 0.30754220485687256, -1.0510659217834473, 0.9457291960716248, 1.3918017148971558, -0.618164598941803, 0.29106950759887695,
                             0.04303515702486038, -0.8536624908447266, -0.2840697169303894, -1.0496186017990112, -0.06474734097719193, 1.1007181406021118, -0.16007000207901,
                             -0.5384657979011536, 1.0438283681869507, 0.36990296840667725, -0.4687243700027466, 0.4814257025718689, -0.6147325038909912, -0.7100877165794373,
                             -0.2518196105957031, -0.284336119890213, -0.29171696305274963, -1.9104725122451782, 0.4549412131309509, -0.2757413387298584, 0.09433843195438385,
                             -0.9758457541465759, -0.9958629608154297, -0.5669455528259277, 0.7963845133781433, -0.019340481609106064, -0.36204180121421814, -0.1518603414297104,
                             -0.6831619143486023, 0.29337552189826965, -0.10384807735681534, -0.5860868692398071, 0.26557356119155884, 0.10635165870189667, -0.9236822128295898,
                             0.46039456129074097, -0.050529152154922485, -0.41403815150260925, 0.2986096739768982, 0.603757917881012, -0.1598687767982483, -0.3645801544189453,
                             -0.3733322024345398, 1.2351763248443604, -1.724015474319458, 0.2887261211872101, -0.23363347351551056, -0.2533142566680908, 0.35519132018089294,
                             -0.32054343819618225, 0.020851315930485725, -0.02741216868162155, 0.14634332060813904, -0.4177318215370178, 0.6654576659202576, 0.5645134449005127,
                             -0.3884005844593048, -1.1183202266693115, 0.11553452163934708, -0.3306879699230194, 0.7676892876625061, -0.8499392867088318, 0.05281652510166168,
                             -0.1557256281375885, 0.20749057829380035, 1.4386547803878784, 0.6958158612251282, -0.19550922513008118, 0.3894248902797699, -0.20726819336414337,
                             0.2062394618988037, -0.6497825384140015, 0.17426562309265137, 0.05923593044281006, 0.6852697730064392, 0.14470091462135315, 0.11595350503921509,
                             0.46969783306121826, -0.13629184663295746, -0.15272541344165802, -0.01200782135128975, -0.6672297716140747, -0.935870349407196, 0.08753477782011032,
                             0.07495712488889694, -0.37996888160705566, -0.37382110953330994, -0.26308673620224, -0.06988656520843506, -0.9956288933753967, 0.22484593093395233,
                             0.10423411428928375, 0.0652785673737526, 1.0540612936019897, 0.9698362350463867, 0.046748049557209015, -0.9015222787857056, -0.4186868965625763,
                             0.6330364346504211, -0.6691665649414062, 0.10714174807071686, -0.07823530584573746, 0.1919396072626114, 0.5876159071922302, -0.30359113216400146,
                             0.34422171115875244, -0.35873737931251526, 1.1258420944213867, -0.2879641056060791, -1.1132158041000366, 0.3020358383655548, -0.23613572120666504,
                             0.9395972490310669, 0.17532886564731598, -0.3456803560256958, -1.0783451795578003, 0.26779821515083313, 0.9098984599113464, 0.613791286945343,
                             -0.335294634103775, 0.9042884707450867, -0.6692565083503723, -0.43846869468688965, -0.30142292380332947, 1.0211962461471558, -0.7418357133865356,
                             -1.0520809888839722, -0.7527135014533997, -1.6785790920257568, -0.17243662476539612, -0.6673782467842102, 0.5787273049354553, -1.3581695556640625,
                             1.3380060195922852, -0.5421662926673889, -0.5725323557853699, 0.5814513564109802, -0.3901490569114685, -0.5192561149597168, -0.7660967111587524,
                             0.19631870090961456, -0.1668502241373062, 0.025263985618948936, 0.14908535778522491, -0.45916181802749634, -0.05947578325867653, -0.24488626420497894,        
                             -0.9397898316383362, 0.41701367497444153, -0.46659722924232483, -0.272495836019516, 0.38524624705314636, 0.8962149024009705, -0.3244347870349884,
                             -0.2850518524646759, 0.6528770923614502, 0.03237779811024666, 0.04364566504955292, -0.24373994767665863, 0.395347535610199, -0.7772626876831055,
                             -0.24284960329532623, 0.1775265485048294, -0.0561564639210701, -0.2060232162475586, 0.36921918392181396, 0.5344421863555908, -0.5555955767631531,
                             1.2276276350021362, -0.07060407102108002, -0.6184007525444031, 0.18186354637145996, -0.03577045351266861, -0.9013174176216125, -0.10277583450078964,
                             0.2911648452281952, 0.3711414337158203, -0.4666562080383301, -0.026299014687538147, 0.35221409797668457, -0.1007944792509079, -0.2579173147678375,
                             -0.3868344724178314, -0.12725263833999634, 0.2156306952238083, 0.025716502219438553, 0.6678427457809448, 0.32845714688301086, 0.5251978635787964,
                             0.010467594489455223, 0.7585484385490417, 0.5791895389556885, 0.8195014595985413, 0.30264902114868164, -1.1879596710205078, 0.5044686794281006,
                             0.4443904161453247, 0.0756133571267128, 0.2822660207748413, -1.3564269542694092, -0.6851717829704285, 0.5234954953193665, -0.0993247926235199,
                             -1.1297029256820679, 0.43998563289642334, 1.0485740900039673, 0.49757999181747437, -0.6866296529769897, -0.059330932796001434, -0.7649421691894531,
                             0.03725295886397362, -1.4082890748977661, -0.13200980424880981, -0.5960607528686523, -0.46805471181869507, -0.7205100059509277, 0.1580548882484436,
                             -0.8534887433052063, -0.8475067019462585, -0.2271936982870102, 0.1548040360212326, -1.120784044265747, 0.2824884355068207, -0.07575475424528122,
                             0.2883884608745575, -1.1170003414154053, -0.003763637039810419, 0.2813403010368347, -0.24604451656341553, -0.1474195271730423, -0.34635990858078003,
                             -0.2382298856973648, -0.48834049701690674, -1.0098447799682617, -0.12091445177793503, 0.0341649167239666, -0.26298871636390686, 0.08548253029584885,
                             0.17214414477348328, 0.28713297843933105, -0.7907886505126953, 1.5293896198272705, 0.17119064927101135, -0.029858862981200218, 0.5484588742256165,
                             0.24890199303627014, -0.41645416617393494, -0.3899105489253998, -0.38388752937316895, 1.2328739166259766, -0.2900119423866272, 1.0320820808410645,
                             -0.2992490231990814, 0.5868529677391052, 0.432228147983551, 0.4087528586387634, 0.5696731209754944, -0.1527819037437439, 0.230552077293396,
                             -0.6769532561302185, 0.318988561630249, -0.10196435451507568, 0.24247993528842926, -0.04875825718045235, -0.1017599105834961, -0.055065810680389404,
                             0.2671518921852112, -0.2240642011165619, 0.3527427017688751, -0.3763459622859955, -0.23884417116641998, -0.9940579533576965, 1.5508801937103271,
                             -0.518191933631897, -0.35862696170806885, -0.7277429699897766, -0.5685166120529175, -0.3493131995201111, -0.7227171659469604, 0.826164722442627,
                             -1.48738694190979, -0.36358222365379333, 0.382546067237854, 0.6147486567497253, 1.0588020086288452, -0.1963508129119873, 0.04238341748714447,
                             -0.5542107224464417, -0.14205528795719147, 0.35498863458633423, -0.3419051766395569, -0.6902903318405151, -0.45527055859565735, 0.29941150546073914,
                             0.3161713480949402, 0.7020865678787231, 0.11513549089431763, -0.34040701389312744, 0.05898216366767883, -0.551433265209198, 0.2575165331363678,
                             -1.0826705694198608, -0.7885127067565918, 0.1492818146944046, -0.2733878195285797, 0.5616562366485596, -1.0599734783172607, -0.11597935855388641,
                             0.8625807166099548, 0.48389896750450134, -0.02261841669678688, 0.20263880491256714, 0.495903879404068, 0.7434864640235901, 0.16030637919902802,
                             0.34625232219696045, -0.17486900091171265, 1.171386480331421, -0.028636235743761063, 0.7785208225250244, -0.2788718044757843, -0.48836007714271545,
                             0.11436251550912857, -0.4676758646965027, -0.6700294613838196, -0.042981408536434174, -0.02089627832174301, 0.22628383338451385, -0.9242793321609497,
                             -0.5039559006690979, 0.23541642725467682, -0.5525899529457092, 0.4830001890659332, 0.8486968874931335, -1.1026595830917358, -0.5127615332603455,
                             -0.9574904441833496, 0.8550878167152405, 0.8443751335144043, -0.09472949802875519, 0.1417018622159958, 0.6591985821723938, -0.35131213068962097,
                             0.2790352404117584, 0.5552213191986084, 0.43340784311294556, 0.06822849810123444, -0.7842282056808472, 0.5126801133155823, 0.04218064621090889,
                             -1.0505990982055664, 0.1843569278717041, -0.04959290474653244, 0.8098521828651428, -0.7585306763648987, 0.19168637692928314, 0.3446279466152191,
                             -0.6822971105575562, 0.11177827417850494, -0.6731106042861938, -0.4982883036136627, -0.15066677331924438, 0.6000230312347412, 0.4990248680114746,
                             0.5820041298866272, -0.23437927663326263, -0.3325900435447693, 0.4024834632873535, -0.09581796824932098, 0.10312812030315399, 1.0381823778152466,
                             1.0384525060653687, -0.6481770873069763, 0.024986643344163895, 0.3518242835998535, 0.023964084684848785, 0.6307371258735657, 0.060541339218616486,
                             0.16403892636299133, -0.37779125571250916, 0.2991880774497986, -0.5911096930503845, 0.3429447412490845, -0.1659044623374939, 0.2431207001209259,
                             -0.3907628655433655, 0.30586445331573486, 0.3974452614784241, -0.4343331754207611, 0.21670660376548767, -0.010783001780509949, 0.40646594762802124,
                             1.1520366668701172, 0.9049986600875854, 0.22155334055423737, -0.05723953619599342, 0.6275542378425598, 0.5207217335700989, 0.2611238360404968,
                             -1.0924724340438843, -0.4274524450302124, -0.44185054302215576, -0.9447586536407471, -0.911583423614502, -0.7004214525222778, 0.32973283529281616,
                             0.028617899864912033, -0.3384590446949005, 0.4385335445404053, 0.04989173635840416, 0.5630132555961609, -0.9253649115562439, -0.3391704261302948,
                             0.33903998136520386, -0.3664064109325409, 0.08783450722694397, 0.6845093965530396, -0.6204856634140015, -1.068819522857666, -0.06905335932970047,
                             -0.08615623414516449, -0.1230766549706459, 0.11973412334918976, -0.8462302088737488, 0.23326441645622253, -0.8821974992752075, 0.21168780326843262,
                             0.7301581501960754, 0.19926422834396362, 0.3549554944038391, -0.46926137804985046, 0.26208731532096863, 0.5104659795761108, 0.22367635369300842,
                             -0.485633909702301, 0.3888915181159973, -0.6423341035842896, 0.437308669090271, 0.21575838327407837, -0.8086127042770386, -0.6134805679321289,
                             -0.2694832980632782, -0.7123908996582031, -0.4393935799598694, 0.36542239785194397, 0.5623076558113098, 0.508285403251648, -0.21585524082183838,
                             -0.4785059690475464, 1.053848147392273, -0.8788537383079529, 0.31482401490211487, 0.7134915590286255, -1.1699490547180176, 0.6162691712379456]

In [ ]:
pipeline_plano_individual = [

    {
        "$vectorSearch": {
            "exact": True,
            "index": "vector_search_index",
            "candidates": 200,
            "limit": 50,
            "path": "embedding_text",
            "queryVector": queryVector,
            "filter": { "doc_type": "markdown" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_type": 1,
            "source_file": 1,
            "text": 1,
            "score": { "$meta": "vectorSearchScore" }
        }
    },
    { "$sort": { "score": -1 } },
    { "$limit": 5 }, 
    
    {
        "$unionWith": {
            "coll": "multimodal-rag", 
            "pipeline": [
                {
                    "$vectorSearch": {
                        "exact": True,
                        "index": "vector_search_index",
                        "candidates": 200,
                        "limit": 50,
                        "path": "embedding_text",
                        "queryVector": queryVector,
                        "filter": { "doc_type": "image_text" }
                    }
                },
                {
                    "$project": {
                        "_id": 0,
                        "doc_type": 1,
                        "source_file": 1,
                        "text": 1,
                        "score": { "$meta": "vectorSearchScore" }
                    }
                },
                { "$sort": { "score": -1 } },
                { "$limit": 25 } # Top 3 image
            ]
        }
    }
]


results = mongo.collection.aggregate(pipeline_plano_individual)
# print the results
pprint(list(results))

[{'doc_type': 'markdown',
  'score': 0.8667113780975342,
  'source_file': 'D:\\OneDrive - Instituto Tecnologico y de Estudios '
                 'Superiores de '
                 'Monterrey\\Github\\MultimodalRAG\\data\\Edificio '
                 'Tello\\GENERATED KNOWLEDGE BASE\\raw_extraction.md',
  'text': "2. el concreto tendr una resistencia de f'ci=210kg/cm2 al momento "
          'del tensado\n'
          '3. el slump mximo podr ser superado por medio de la utilizacin de '
          'aditivos plastificantes o superplastificantes. 4. el diseo de '
          'mezcla deber ser tal que se evite la segregacin y el sangrado. **3. '
          'tertiary element: parmetros sismo resistentes norma e.030 (seismic '
          'resistance parameters standard e.030)**\n'
          '- **description**: this section outlines the key seismic design '
          'parameters as per peruvian standard e.030. - **content '
          'breakdown**:\n'
          '- **sistema estructural sismorresistente*

: 